## Levatamos Fermionic

In [35]:
import numpy as np
import openfermion as of
from tqdm import tqdm
from itertools import combinations
from openfermion.utils import commutator, count_qubits, hermitian_conjugated
import functools
import concurrent.futures
from numba import njit
import tensorflow as tf
import scipy
import sparse
import itertools
import linecache
from multiprocessing import Pool, cpu_count

# Generación de base
class fixed_basis:
    @staticmethod
    def int_to_bin(k, d):
        return np.base_repr(k, 2).zfill(d)

    @staticmethod
    def bin_to_op(b):
        tups = [(i, 1) for i, k in list(enumerate(list(b))) if k == '1']
        return of.FermionOperator(tups)

    def idx_to_repr(self, idx):
        return self.canonicals[idx]

    def opr_to_idx(self, opr):
        for i in range(self.size): # Evitar esto ordenando opr
            if self.base[i] == opr:
                return i

    # Calcula el valor medio a partir del indice del vector y el operador
    def idx_mean_val(self, idx: int, op: of.FermionOperator):
        vec = self.idx_to_repr(idx)
        return np.real(np.transpose(vec) @ of.get_sparse_operator(op, n_qubits=self.d) @ vec)

    # Calcula el valor medio a partir de un estado y el operador
    def mean_val(self, vec, op):
        idx = self.opr_to_idx(vec)
        return self.idx_mean_val(idx, op)

    # Calcula la contracción de un operador sobre dos estados dados
    def idx_contraction(self, idx_1, idx_2, op):
        rep = lambda x: self.idx_to_repr(x)
        return np.real(np.transpose(rep(idx_1)) @ of.get_sparse_operator(op, n_qubits=self.d) @ rep(idx_2))

    def create_basis(self, d, num = None, pairs = False):
        basis = []
        num_ele = []
        for k in range(0,2**d):
            b = self.int_to_bin(k, d)
            if num != None:
                if b.count('1') == num:
                    if pairs:
                        if np.all(b[::2] == b[1::2]):
                            oper = self.bin_to_op(b)
                            basis.append(oper)
                            num_ele.append(k)
                    else:
                        oper = self.bin_to_op(b)
                        basis.append(oper)
                        num_ele.append(k)
            else:
                oper = self.bin_to_op(b)
                basis.append(oper)
        return basis, num_ele

    def __init__(self, d, num = None, pairs = False, basis = None, num_ele = None):
        self.d = d
        self.num = num
        self.m = num
        # Si nos da la base, la levantamos (asumimos GC). Si no, la creamos
        if basis is None:
            self.base, self.num_ele = self.create_basis(d, num, pairs)
        else:
            self.base, self.num_ele = basis, num_ele
        self.size = len(self.base)
        self.canonicals = np.eye(self.size)
        self.pairs = pairs

    @staticmethod
    def cdc(i, j):
        return of.FermionOperator(((i,1),(j,0)))

    @staticmethod
    def cc(i, j):
        return of.FermionOperator(((i,0),(j,0)))

    # Del indice, cuenta el número de partículas
    def num_idx(self, idx):
        b = self.int_to_bin(idx, basis.d)
        return b.count('1')

    # Calculo de rho1 (via directa, lento, y solo definido en la base por ahora)
    def rho_1(self, op):
        # Necesitamos un índice, es?
        if type(op) != int:
            op = self.opr_to_idx(op)
        mat = np.zeros((self.d, self.d))
        for i in range(self.d):
            for j in range(self.d):
                cdc = self.cdc(j, i)
                mat[i,j] = self.idx_mean_val(op, cdc)
        return mat

# Calculo de generadores de rho1
def rho_1_gen(basis):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    indices = []
    values = []
    shape = (basis.d, basis.d, basis.size, basis.size)
    d = basis.d
    for i in tqdm(range(0, d)):
        for j in range(0, d):
            # Generamos el operador
            op = basis.cdc(j, i)
            #print(op)
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# Calculo de rho1 (via generadores) de un vector en la base canonica
def rho_1(vect, rho_1_arrays):
    if len(vect.shape) == 1: # vectores
        return sparse.einsum('k,ijkl,l->ij', vect, rho_1_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad
        return sparse.einsum('ijkl,kl->ij', rho_1_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_1_arrays)

# Calculo de indices de rho2kkbar
def get_kkbar_indices(t_basis):
    indices = []
    for i, ind in enumerate(t_basis.num_ele):
        v = t_basis.int_to_bin(ind, t_basis.d)
        if np.all(v[::2] == v[1::2]):
            indices.append(i)
    return indices

# Calculo de generadores de rho2
def rho_2_gen(basis, t_basis, idx_list = []):
    # Vamos a crear un hipersparse de TF, almacenamos los valores acá
    d = basis.d
    indices = []
    values = []
    if len(idx_list) == basis.m:
        idx_list = idx_list
    elif len(idx_list) == basis.m**4:
        idx_list = np.unique(idx_list[:,0])
    else:
        idx_list = range(t_basis.size)
    shape = (len(idx_list), len(idx_list), basis.size, basis.size)
    for i, ii in tqdm(enumerate(idx_list), total=len(idx_list)):
        for j, jj in enumerate(idx_list):
            # Generamos el operador
            op = t_basis.base[jj]*of.utils.hermitian_conjugated(t_basis.base[ii])
            if basis.num == None:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))
            else:
                mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extraemos la información
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([i, j, r, c])
                values.append(v)

    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

# rho_m_gen aux func
def process_chunk(args):
    chunk, m_basis, basis, d, it_set = args
    indices = []
    values = []
    for ii in chunk:
        for jj in it_set:
            # Generate the operator
            op = m_basis.base[jj] * of.utils.hermitian_conjugated(m_basis.base[ii])
            mat = np.real(of.get_sparse_operator(op, n_qubits=d))[np.ix_(basis.num_ele, basis.num_ele)]
            # Extract the information
            n_r, n_c = mat.nonzero()
            data = mat.data
            for r, c, v in zip(n_r, n_c, data):
                indices.append([ii, jj, r, c])
                values.append(v)
    return indices, values

# Parallelized rho_m_gen
def rho_m_gen(basis, m, num_workers=None):
    if num_workers is None:
        num_workers = cpu_count()  # Use all available CPUs by default
    
    indices = []
    values = []
    m_basis = fixed_basis(basis.d, num=m, pairs=basis.pairs)
    shape = (m_basis.size, m_basis.size, basis.size, basis.size)

    it_set = np.arange(m_basis.size)
    chunks = np.array_split(it_set, num_workers)  # Split `it_set` into chunks for each worker

    # Use multiprocessing Pool for parallel processing
    with Pool(processes=num_workers) as pool:
        # Pass arguments as tuples instead of using a lambda
        results = list(
            tqdm(
                pool.imap(
                    process_chunk, 
                    [(chunk, m_basis, basis, basis.d, it_set) for chunk in chunks]
                ),
                total=num_workers
            )
        )
    
    # Collect results from all processes
    for indices_chunk, values_chunk in results:
        indices.extend(indices_chunk)
        values.extend(values_chunk)

    # Construct the sparse array
    indices_t = np.array(indices).T
    s_t = sparse.COO(indices_t, values, shape=shape)
    return s_t

def rho_m(vect, rho_m_arrays):
    return sparse.einsum('k,ijkl,l->ij', vect, rho_m_arrays, vect)

# Calculo de rho2 (via generadores) de un estado en la base canonica
def rho_2(vect, rho_2_arrays):
    if len(vect.shape) == 1: # vectores SOLO RHO2 COMPLETA
        return sparse.einsum('k,ijkl,l->ij', vect, rho_2_arrays, vect)
    elif len(vect.shape) == 2: # mat densidad SOLO RHO2 COMPLETA
        return sparse.einsum('ijkl,kl->ij', rho_2_arrays, vect)
    else: # mat densidad batcheadas
        return sparse.einsum('bkl,ijkl->bij', vect, rho_2_arrays)

# Calculo de generadores de K (usado para quasiparticles) WIP SPARSE
def k_gen(basis):
    mat = np.zeros((basis.d, basis.d, basis.size, basis.size))
    d = basis.d
    for i in tqdm(range(0, d), total=d):
        for j in range(0, d):
            op = basis.cc(j, i)
            if basis.num == None:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()
            else:
                mat[i,j,::] = np.real(of.get_sparse_operator(op, n_qubits=d)).todense()[np.ix_(basis.num_ele, basis.num_ele)]
    return mat

def k_vect(vect, k_gen):
    return np.einsum('k,ijkl,l->ij', vect, k_gen, vect)

# Calculo la matrix rho de cuasipartículas  WIP SPARSE
def rho_qsp(vect, rho_1_arrays, k_arrays, rho1 = None):
    if type(rho1) == None:
        rho1 = rho_1(vect, rho_1_arrays)
    k = k_vect(vect, k_arrays)

    mat = np.block([[rho1, k], [-np.conjugate(k), np.eye(rho_1_arrays.shape[0])-np.conjugate(rho1)]])
    return mat

# Devuelve los indices que tienen a level ocupado
def level_proy(d, level):
    ids = []
    for k in range(0,2**d):
        b = fixed_basis.int_to_bin(k, d)
        if b[level] == '1':
            ids.append(k)
    arr = np.zeros(2**d)
    arr[np.array(ids)] = 1
    return arr, ids

def parity_levels(d):
    rng = range(2**d)
    binary_repr = np.vectorize(np.binary_repr)(rng)
    ones_c = np.char.count(binary_repr, '1')
    return np.array(rng)[ones_c % 2 == 1] # seleccionamos estados impares

# Devuelve el vector postmedido
def measure(basis, vect, level = 1):
    l_arr, l_ids = level_proy(basis.d, level)
    proy_v = vect * l_arr
    comp_arr = np.logical_not(l_arr).astype(int)
    comp_v = vect * comp_arr
    norm = lambda v: v / np.linalg.norm(v)
    return norm(proy_v), norm(comp_v)

def entropy(rho, m):
    S_fun = lambda rho: -1*np.trace(rho @ scipy.linalg.logm(rho)) / np.log(2)
    ent = S_fun(rho) / (np.log2(scipy.special.binom(basis.d, m)))
    return ent

# Levanta bases de QChem
def build_csv_basis(csvf, d):
    # Construimos la base
    ops = []
    with open(csvf, 'r') as basis:
        num_ele = []
        # Contar los niveles
        m_level = 0
        # Creamos operadores
        for l in basis.read().splitlines()[4:]:
            natop = [int(x) for x in l.split(' ')[1:]] # Operador en forma de lista
            #print(natop)
            op = of.FermionOperator(([(i, 1) for i in natop]))
            ops.append(op)
            # Contamos niveles
            m_level = max(m_level, *natop)
            # Determinamos el índice
            natop_to_int = lambda x: np.sum([2**(d-1-i) for i in x])
            num_ele.append(natop_to_int(natop))
 
        # Determinamos m y d
        num = len(natop)
        #assert d == m_level+1

    return fixed_basis(d, num = num, pairs = False, basis = ops, num_ele=num_ele)

Debido al número de niveles (innecesarios) en el core, el objetivo es calcular la base natural (a partir de rho1), y proyectar el fundamental sobre el subespacio generado por los estados no ocupados. Allí, buscamos la descomposición, y luego volvemos a la base original

In [36]:
# Aux func
def remove_null_terms(op):
    op_nn = of.FermionOperator.zero()
    terms = list(op.terms.items())
    act_idx = lambda tt: np.array([i[0] for i in tt[0]])
    filtered_terms = []

    for term in terms:
        acts = np.array(act_idx(term))
        u, c = np.unique(acts, return_counts=True)
        if np.max(c) == 1:
            filtered_terms.append(term)

    return filtered_terms

def op_to_rep(basis, op): # CHEQUEADO
    op = -1*of.transforms.normal_ordered(op)
    terms = list(op.terms.items())
    act_idx = lambda tt: [i[0] for i in tt[0]]
    vect = np.zeros(basis.size)

    for term in terms:
        act = act_idx(term)
        nele = np.sum([2**(basis.d-1-i) for i in act]) # si
        nele_idx = np.argwhere(basis.num_ele == nele)[0][0]
        vect[nele_idx] += term[1]
    
    return vect
    
def build_basis_from_vect(basis, vect, tol = 1e-10):
        ops = []
        num_ele = []
        vect_pro = []
        for idx, coord in enumerate(vect):
                if np.abs(coord) > tol:
                        ops.append(basis.base[idx])
                        num_ele.append(basis.num_ele[idx])
                        vect_pro.append(vect[idx])
        return fixed_basis(basis.d, num = basis.num, pairs = False, basis = ops, num_ele=num_ele), np.array(vect_pro)


In [37]:
from concurrent.futures import ProcessPoolExecutor, as_completed
import sympy as symp

# Dado la base, una rotación entre los 1SP states y un vector (representado en basis)
# devuelve la representación en la base natural
def process_term(term, targ_sp, targ_sp_dict):
    act_idx = lambda tt: [i[0] for i in tt[0]]
    act_idxsp = lambda tt: [key[0][0] for key, value in tt.terms.items()]

    term_levels = []
    for i in act_idx(term):
        ttt = targ_sp[i]
        term_levels.append(act_idxsp(ttt))
    
    combinations = unique_combinations(term_levels)
    partial_vect_op = of.FermionOperator.zero()
    for comb in combinations:
        coeff = np.prod([targ_sp_dict[act_idx(term)[idx]][i] for idx, i in enumerate(comb)])  # Coefficients of targ_sp
        coeff *= term[1]  # Coefficient of the original term
        partial_vect_op += of.FermionOperator(([(l, 1) for l in comb]), coeff)

    return partial_vect_op

# Genera las combinaciones de elementos de la lista, tal que no se repitan los niveles (no nulos)
def unique_combinations(lists):
    all_combinations = itertools.product(*lists)
    valid_combinations = [comb for comb in all_combinations if len(set(comb)) == len(comb)]
    
    return valid_combinations

# Main function, rota el estado a la base dada por C
def natural_basis_rotation(basis, C, vect):
    # Calculamos la transformación de cada generador
    targ_sp = []
    for i in range(basis.d):
        op = np.sum([of.FermionOperator(((basis.d-1-j,1)),C[i,j]) for j in range(basis.d) if np.abs(C[i,j]) > 1e-5])
        targ_sp.append(op)

    targ_sp.reverse()

    # Reemplazamos en el fundamental
    vect_op = np.sum([vect[i] * basis.base[i] for i in range(len(vect))])
    terms = list(vect_op.terms.items())

    # A partir de este diccionario, recuperamos los coeficientes asociados a cada nivel de cada término de \bar{a}^\dag
    targ_sp_dict = []
    for sp in targ_sp:
        targ_sp_dict.append({key[0][0]: value for key, value in sp.terms.items()})

    # Para cada término de fundamental, reemplazamos por los nuevos ops, y calculamos todas la combinaciones no nulas. Luego, sumamos
    vect_op_ft = of.FermionOperator.zero()

    with ProcessPoolExecutor() as executor:
        # Submit tasks to the executor
        futures = [executor.submit(process_term, term, targ_sp, targ_sp_dict) for term in terms]

        # Use tqdm to track progress
        for future in tqdm(futures):
            vect_op_ft += future.result()

    # Calculamos la representación en la base
    ext_basis = fixed_basis(basis.d, basis.m)
    res = op_to_rep(ext_basis, vect_op_ft)
    res = 1/np.linalg.norm(res) * res

    return res, vect_op_ft


### Determinación de estado

##### Cargamos el input

In [46]:
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
# Levantamos la base, y calculamos rho1
basis = build_csv_basis('nh3_basis', 16)
rho_1_arrays = rho_m_gen(basis, 1, num_workers=32)

# Input
rho_1_obj  = np.sort(np.concatenate([np.repeat(1/2, 2*6), np.repeat(1, 2*4)]))
rho_2_obj = np.sort(np.concatenate([np.repeat(1/9, 2*9+9), np.repeat(1/3, 2*6+6), np.repeat(1/2, 2*12+24), np.repeat(1, 2*1+4), np.repeat(2/3, 9), np.repeat(0, 12)]))
vect = np.array([1.243394647249271e-02,-1.840558652407957e-09,1.934930660377851e-09,3.525283290556555e-02,-1.330315103971384e-04,1.153822398916613e-03,-1.477377176693350e-09,-1.535508808958358e-09,-1.220892392840834e-03,-1.441355089197103e-02,-7.668159497551058e-02,4.384034382050232e-03,9.567988165702244e-09,1.199029123275947e-09,1.645071045632985e-01,-1.098465383278331e-02,-1.672457548261973e-02,-1.688154890487127e-09,1.117861938487769e-09,1.736480040013703e-02,1.329648322498869e-04,-1.741257095466118e-04,3.137751479601771e-11,-3.153209423466064e-11,7.537779595079559e-04,4.613911639035038e-04,1.135568128795057e-04,2.272745491707633e-11,7.130673413073194e-12,3.062862760938863e-04,8.312712074886758e-04,-1.511802477979973e-10,1.692255696872807e-10,1.844187045768434e-03,-1.443114584778927e-10,2.995558981852082e-06,-4.934099967203187e-07,4.185348944981315e-13,9.015999387643671e-13,2.715081684861054e-06,1.510414870046984e-06,1.323612387129778e-06,2.914141622589710e-12,2.658330114449096e-12,2.274390402729312e-06,-2.419378190007803e-07,-4.204612779083703e-13,2.663778685279287e-12,1.561264637712501e-06,-3.501568014208307e-12,2.842415187644858e-08,-6.361815169522173e-13,1.808853395614499e-12,-4.429358265344314e-08,3.128135999443146e-12,1.642583360534204e-12,2.130230325548791e-09,-1.358162909242080e-03,4.453606398608784e-04,-1.012697437676362e-09,4.466179242447087e-10,1.893690385766987e-09,-1.619178487119328e-03,-1.513307773187834e-04,1.749984458567954e-09,-2.152395886028246e-10,6.267511609697847e-10,-1.745238329329890e-09,-1.732357908665901e-01,6.686532328624404e-04,-8.841352948393908e-09,2.635786284657351e-09,1.715832788082911e-10,-1.362926018455597e-02,-1.555653276765697e-02,-1.180741797724781e-09,5.356006011380177e-11,4.095876863258496e-11,-6.449696808942645e-05,2.086754630305033e-05,-1.331796914167905e-10,1.993335710651183e-11,-3.808496596325881e-11,3.402643470793605e-05,-8.272159340754704e-05,-1.691388079085749e-10,1.051579282386724e-11,-3.067772731812731e-03,-1.737154758318009e-03,-1.827869177844987e-10,1.714939268595484e-04,-5.187286371779722e-13,3.279613062018089e-13,2.800769026079951e-06,2.889471129734264e-08,7.001661683608277e-13,2.398785298666155e-12,6.457685490656122e-13,6.454964112068505e-07,-2.824032604710974e-07,1.454973787502274e-12,-1.966492825532069e-13,-1.931468133338249e-05,-3.952902353957050e-07,-1.747761736413366e-12,2.703206835478224e-06,-3.272107723558087e-13,-1.095529667971930e-06,8.016649374281332e-08,-2.836447040903605e-12,1.354302524445787e-07,9.749717314017024e-08,2.181305351203513e-10,4.453602930150678e-04,-1.003207694231023e-04,2.381196474065216e-10,1.777209796376924e-09,-4.209807677041380e-10,-2.279994769016543e-02,-2.048933684720278e-04,-1.668762353582566e-10,1.709908713846541e-10,-7.893460729398138e-10,1.346431404992350e-09,1.185264422019380e-02,-3.818100124718310e-04,2.713835840082960e-09,-2.512296511254558e-10,1.122008836049510e-09,-1.627400188195570e-01,1.134169014679196e-02,-1.591128945840484e-09,-1.498368340779704e-10,9.477329555927955e-12,2.776252517515760e-04,-4.311757480825276e-06,4.349140131117555e-10,9.109412405655812e-11,1.165845084917071e-12,-5.438101093455635e-04,-2.391065352312614e-04,2.978460055405506e-10,-3.128242528761478e-11,-1.779797814977344e-03,2.671918026244706e-04,5.637668989369656e-11,-2.038736480875317e-03,-7.241001813299179e-14,-7.145969494875754e-14,2.666621321840253e-06,1.219730464903507e-08,-2.733047217468337e-13,-5.003821204769332e-13,-6.811579815279453e-13,-1.152122014705039e-06,-2.253077326131922e-07,-2.215147161600015e-12,-1.927491873236668e-13,-1.934433636494027e-05,1.963138799842181e-06,-1.529317501131918e-12,-2.593101689421563e-06,-1.123847166623873e-13,5.892680642891769e-08,-2.788936153204453e-07,1.195058732875733e-12,2.001535174476491e-08,1.833075364944863e-06,3.525283209806816e-02,2.402316931333768e-10,-4.870820180696072e-10,-7.209552458849861e-03,-3.982873399657169e-04,-8.386396286689735e-03,2.282743696033730e-11,6.058990222328040e-10,2.381537509486755e-05,6.739122359300584e-04,-3.377428929321815e-01,2.283729634857835e-02,-3.332160659514146e-10,-2.265833320966890e-09,-5.076572607463802e-02,3.375057661658404e-03,-1.298186863004271e-01,1.012717965298797e-10,-4.373992112970925e-10,1.111931205892184e-02,-8.182750264080104e-04,-1.837662381900288e-04,-2.875709434598367e-10,6.573591700346820e-11,-1.601368175058160e-04,-6.797641134305554e-05,-1.548026281555003e-04,2.218457570961118e-10,4.332136662870872e-11,-1.834296624733876e-05,1.801543463250093e-03,-1.210906584205500e-10,2.996553474957191e-10,-3.809891615808586e-04,-1.392037964362899e-11,8.567691475925646e-08,1.156206650438983e-06,1.379539361597378e-13,-1.815796298127014e-13,-1.633869445149719e-07,-4.000517857280034e-08,1.404670376630637e-06,-6.767456855480698e-13,-2.240565809274942e-13,-3.770603792019889e-07,-1.738778173147339e-05,1.285922612449092e-12,1.405725971086499e-12,1.625702485775422e-06,-3.908577034205303e-13,-6.553698735394200e-08,-9.399864431032529e-13,2.093734527562763e-13,9.915817780318884e-09,6.837484023264274e-13,4.885017639813363e-13,-1.330320229048943e-04,1.954891917352925e-09,4.522487297329274e-09,-3.982854499322887e-04,-2.484541667418509e-04,-1.323285074961264e-03,5.280732456179157e-10,2.061529061715376e-09,6.325981261269623e-04,3.236340245108640e-03,-6.215312272088013e-03,-4.856036178436107e-02,-1.511421119699829e-09,-1.632894202218423e-09,1.359766334680235e-02,1.596936153388291e-01,-5.496085434855112e-03,-4.839303363189402e-10,-2.882008080724041e-09,-1.190304345047720e-02,5.481894843853779e-05,2.215680863423064e-04,-1.916527907266814e-11,7.594505849927200e-11,-4.574998064268897e-05,1.239340055025616e-04,3.021690931247345e-05,-4.521928386818023e-11,1.173984128493749e-11,1.110758946458989e-05,-8.513272488502695e-04,-1.366280468468190e-11,1.694630805539131e-11,-2.757479432786068e-03,-1.149304937038182e-11,5.533724456651676e-07,2.816293645119842e-06,6.726076241905340e-13,-4.042853713278752e-13,-5.120300817006361e-07,-2.162907927581098e-06,4.453281430114497e-07,1.034320653019660e-12,9.821434168207359e-14,5.021016193946197e-07,-5.213523713763654e-06,5.730857796462333e-14,7.135338581206196e-13,-1.867894044257419e-05,-1.258111786086302e-15,-2.729296131769249e-07,3.971582569357783e-12,-9.062512763749353e-13,-1.044432559936736e-06,2.943770795737393e-12,1.056220366495767e-13,1.153819934655282e-03,8.926290763143838e-10,-2.227231457335323e-09,-8.386396311454297e-03,-1.323284245295588e-03,-1.097163111467018e-02,7.841749127233278e-10,1.426497822121814e-09,2.832415065774873e-03,3.129108542448462e-02,-3.183578803546437e-02,4.378029667539920e-03,5.936998888294316e-10,2.102051913674205e-09,1.312551850567207e-02,-1.127675386338926e-02,-6.918784432655255e-02,-2.606983257540796e-09,-8.016347867774242e-09,-1.492766572638106e-01,1.080114425890831e-04,3.981360654604570e-05,8.046625606233056e-12,5.124037928924480e-11,-1.047304982988126e-04,-4.075231274192153e-04,9.416840924209100e-05,-4.443117620561883e-11,-1.556614742308470e-11,-7.394982650806578e-04,2.357497078527396e-05,-2.477884505044263e-10,-9.464516462266168e-11,-1.165932770185502e-03,-1.062701143606709e-10,1.492678275145358e-06,5.757654955016665e-07,-2.692700466132964e-12,-6.716910625390454e-13,-9.815954950477175e-07,-3.203487246265762e-06,2.349121133575738e-06,2.079039982379104e-12,-8.567781408156293e-13,-3.243058260910526e-06,-4.025539498933565e-06,-3.134931259010352e-12,-4.541358672596622e-13,-1.455434943021480e-05,-1.512336588412109e-12,3.916946178975445e-08,9.606569335800958e-13,-8.609074617916674e-13,8.428736083699116e-08,-3.876122549858348e-12,-1.338469386591558e-12,-1.136824625066332e-09,-1.619178960299634e-03,-2.279994741002105e-02,1.763694878580867e-10,6.355427306988780e-10,1.528360290504544e-09,-4.293119082271651e-05,-4.039239002398824e-04,-1.427431357849757e-11,2.475770703071500e-10,-3.034684484182435e-09,-9.390869019454888e-09,8.156939912752479e-04,-1.639329005922855e-01,1.862769509300856e-09,-1.054099869794759e-09,1.934661841510905e-09,-3.663054074914590e-04,-6.441745155223946e-04,-1.962229303635955e-09,-6.131847264575863e-11,-3.530287413026408e-11,3.128506403634826e-05,-5.349503148537355e-04,1.265411945383156e-10,-1.834325371765364e-10,1.349316038491572e-10,-4.379748675935452e-06,-3.336997937735023e-05,1.641174841000940e-10,1.939066674709126e-10,-2.737597923183614e-04,1.339021468941212e-05,-7.864336247833713e-11,-2.634743971808382e-04,2.688126582756216e-12,2.559829576914372e-12,1.998095622492191e-07,-1.093089288968400e-06,-2.445308923709329e-13,1.856968398647397e-12,-2.295599940667356e-12,-3.208212372964785e-08,-2.829608915587329e-07,-1.892055148989369e-13,4.109363455381043e-12,-2.295698164239726e-06,-5.326844754237444e-07,-5.222070229849164e-13,-2.430018945106653e-06,3.657951740053755e-13,1.393296638732139e-08,2.460363192940941e-07,-9.880057759988639e-13,-1.118575470071655e-08,-1.624134763511588e-06,-1.374537025995851e-09,-1.513282722145976e-04,-2.048939794001317e-04,3.235100558905889e-10,4.795388290034784e-10,-1.911094727893206e-09,-4.039244070171918e-04,1.282646895515069e-03,-1.462831093441758e-09,-2.923557172684728e-10,-9.610049680067099e-10,-1.292690441228629e-09,-1.674941568091919e-02,7.564377021794115e-04,-6.415127829294263e-10,-2.172291015923063e-09,1.801526146235944e-09,1.276749060656108e-02,1.572472737157845e-01,8.111396353802346e-09,2.687023198337739e-11,-2.768892531065990e-11,-7.386147402159605e-05,-1.292685200536665e-05,-1.564636094980694e-10,1.114413580321723e-10,-3.630967050887986e-12,-4.042566821750334e-05,1.763441575954524e-04,-2.567065647365977e-10,1.679010659132481e-11,-3.736571903522648e-05,1.370777064541066e-03,1.071118792601120e-10,-2.535652694553388e-03,-2.405446405627415e-13,3.160208019470947e-13,-2.233716002787418e-07,-1.471008900078649e-07,-6.261514866244500e-13,-2.581610544896622e-12,4.495316851573048e-13,-5.019744530960630e-07,-1.484081774092322e-06,1.456216296985123e-13,9.909873240498673e-14,-5.745855098019846e-08,1.623040656503330e-05,1.290446255351705e-12,-1.645905715557878e-05,1.503003138165353e-13,-1.391973706535109e-07,2.102566273598402e-08,1.371182216801593e-12,-1.024361978871341e-06,-1.259683281970950e-07,-1.220892473867065e-03,1.708217079684418e-09,9.521846961854762e-10,2.381568448673434e-05,6.325971022957264e-04,2.832414934943847e-03,1.036045270143524e-10,-1.050428844279425e-09,1.193006462844367e-04,5.796965409177317e-04,2.722692479066441e-02,1.585007350452877e-01,-1.891497932622212e-09,9.651861334702849e-09,3.363761629332844e-03,3.248613477501756e-02,2.467053264175018e-02,7.343083230455017e-10,1.220765993009112e-09,-3.021546092953590e-03,-7.977620993020922e-05,1.327936535152756e-04,-1.820467339718415e-10,-1.371803618515722e-12,-1.374894009790027e-05,2.077161310343364e-05,-5.153335726303584e-05,-5.129348180610918e-11,-2.193511286286713e-10,5.860127733818262e-06,2.891607657308106e-03,-6.671117960337288e-11,4.930107841468350e-11,-6.145107099535665e-04,4.320852456817745e-11,-1.157525602427245e-06,-2.103871417273330e-06,2.722226502716138e-13,2.115134775257459e-12,-1.766667198448723e-07,-4.390466342520303e-07,-1.004075739248773e-06,3.119238163079567e-13,1.291234380096442e-12,1.581644116605615e-07,2.132469306763984e-05,1.698137379216518e-12,-3.610089106227271e-12,-3.650280509508379e-06,-2.056289340386711e-12,1.040663038381401e-06,7.181414431376606e-13,1.302837566655445e-12,-1.915857629954129e-07,2.902294936451899e-13,-1.609865351462438e-12,-1.441355180281712e-02,-3.730476872256423e-10,-4.508239552592424e-10,6.739122577260071e-04,3.236338332691535e-03,3.129108568266882e-02,1.434852268394324e-11,9.808524765209899e-10,5.796962657491678e-04,6.477168427791953e-03,1.926149538202827e-01,-2.261844537152953e-02,5.936204757758533e-10,-2.371486268965783e-09,2.136682333331109e-02,-3.481455356763371e-03,3.065239292770708e-01,-4.532772228112601e-10,-1.070109963526679e-09,-4.661211122749512e-02,5.129416870979323e-04,-1.684164105662146e-04,-4.067630903239635e-12,-2.138633817638582e-10,3.911463973092128e-05,-5.926283893978508e-05,9.158419890751891e-04,-2.796987634285151e-10,-2.203318643233873e-10,-1.685143726988334e-04,4.979728096185905e-04,7.383543335697072e-11,-4.021617131060997e-10,-1.697260372581193e-04,7.041446062719296e-12,3.939003932403021e-07,-2.978181781870222e-06,-9.921010143468810e-13,9.952831795883933e-13,-6.198517458735402e-08,-6.291725444923559e-07,1.758986837412485e-06,1.313802549127445e-12,1.008243397588468e-12,-4.301815754997119e-07,2.819165327228908e-05,3.070333386815904e-13,4.395803495331851e-12,-4.327009393534734e-06,-8.612366085380809e-13,-4.325109031377212e-08,-1.022209288893974e-12,-2.104490702605781e-14,1.039117506530898e-08,3.159483456526536e-13,5.425389319135252e-13,-7.668159571234134e-02,-1.524686779784304e-10,4.875226372716576e-09,-3.377428957523410e-01,-6.215312641254656e-03,-3.183578749475664e-02,6.851715424337051e-10,8.730303435794452e-10,2.722692434783210e-02,1.926149505310149e-01,-6.060435681755278e-03,4.198302316434267e-04,-2.676619388589011e-10,-2.049286340508774e-11,1.924078261694486e-02,-1.452562259614318e-03,5.110370643708889e-04,-3.275750066719341e-10,-1.925248614920539e-10,7.919901086363096e-03,1.516950400533361e-03,1.176076331558232e-03,2.048080949157000e-11,2.544419747843837e-11,-6.332132527064748e-03,-5.191914849274689e-03,-4.965529593993454e-04,5.207841004915955e-11,2.244516663760894e-10,-3.148494420804312e-03,5.992816912239786e-05,-3.367021372841768e-12,1.080758788822236e-10,2.251580351676555e-04,-1.501446199203886e-11,8.809960658443853e-06,3.880670699154563e-06,8.948358391661288e-13,5.295010152581146e-14,-4.119854644762004e-05,-1.975009758746402e-05,-3.285010707995731e-06,9.782725319134169e-13,4.190062324332162e-13,-2.299289490563678e-05,3.184773278421996e-09,-8.070560064816189e-13,3.087026119778873e-13,6.793150849136651e-07,-1.078064576467403e-12,3.734626322493130e-08,4.440220672128062e-14,9.754204429774333e-13,1.842449444748021e-07,-2.682716300391537e-12,-4.539676275612114e-12,4.384034138836108e-03,-1.017840003277485e-09,-2.105512091815059e-09,2.283729768593894e-02,-4.856036254167389e-02,4.378028777805940e-03,5.554808208601271e-09,-1.434494972202326e-09,1.585007364034611e-01,-2.261844412034740e-02,4.198310025569768e-04,-1.465136408524632e-04,-7.321733405141035e-10,8.508622674394976e-11,-1.320528496142444e-03,4.758723803900931e-04,-1.657974706486116e-04,-7.876707060526230e-10,-5.468667161010570e-10,-8.854225988772091e-04,3.448049402545814e-04,-1.069674709565659e-04,1.348985389663332e-10,3.146328397655229e-11,-1.469033996314015e-03,5.341387787253372e-04,7.416404165143007e-04,1.119288359118458e-11,1.210033088030708e-10,2.111283953695077e-03,3.753492468503758e-05,-5.018931909228130e-11,-2.145096477144113e-11,7.631033247388898e-05,-9.296885537561901e-11,4.122608102353498e-06,-8.009809068933461e-07,-2.279609259757613e-12,-1.097682763475645e-12,-1.711628441384254e-05,3.944594032539436e-06,1.686342079650413e-06,1.879475360212053e-12,1.589002350779999e-12,3.100092784662992e-06,3.589560506545414e-07,-4.767637678691476e-13,-2.827445112819507e-13,6.859877174694503e-07,-2.691695919827871e-13,4.482013321579508e-07,1.245245047219436e-12,1.984816956101217e-12,1.801911083090299e-06,-5.645727604341204e-12,2.522803950355289e-12,-9.083202254748187e-09,-1.732357904024792e-01,1.185264421749015e-02,9.664875316551197e-10,1.751432627180511e-09,-1.349143450317251e-09,8.156959725111194e-04,-1.674941493973515e-02,-1.422087302537570e-09,1.055265102508067e-10,3.624906639547329e-11,7.217670549455988e-10,-1.448151750024992e-03,1.320360242036256e-04,-2.024015205250980e-09,8.201072683166425e-10,-8.352714571478661e-11,2.188616276903324e-04,-1.536870733402676e-04,2.692560628610908e-10,-1.124467450373871e-11,1.416253580227587e-11,-7.795913819021770e-05,1.878693927891362e-03,-7.753751293352901e-11,1.893951028896916e-10,-1.538513213419535e-10,-1.521709485945315e-04,7.536188968702993e-06,-5.216702188230919e-11,4.812851098589404e-11,3.712926580179901e-04,-3.928083798851533e-05,1.098068015015070e-11,8.669330082162003e-05,-1.056229695072237e-12,3.127014907436435e-12,-1.496193388915883e-06,1.072545579928793e-06,2.010014562418038e-12,-5.041540175879668e-12,-3.025433778363788e-12,-1.496012770032076e-06,-3.543209546118137e-07,1.832529399231823e-12,4.859025947746997e-13,3.154196194540970e-06,-2.158136497102387e-07,4.120597297217210e-13,7.698431659907869e-07,1.045193186860153e-12,1.704470671350110e-06,-9.029037053477685e-08,-2.535244940012423e-12,-2.114832248422118e-07,-3.269004536252669e-08,7.454195116968376e-10,6.686537870708350e-04,-3.818103594862256e-04,1.213338226481993e-09,1.027588844943352e-09,5.755713272998496e-10,-1.639329013761959e-01,7.564380460834798e-04,-5.157140786324385e-09,1.288944161701544e-09,-5.128253293466793e-12,2.475491844295850e-12,1.320378062971901e-04,2.716855689630153e-05,4.939446006013408e-10,-5.154349487241431e-10,5.446789758859264e-11,1.286639689875871e-02,-4.300184342962201e-04,1.110015547914282e-09,7.148829475746382e-11,-8.681302819359297e-11,1.921337348983798e-03,-1.998422838661771e-05,1.004512497413502e-10,-6.374794453917249e-11,-5.941626452114818e-11,-2.927098595766207e-03,-1.788352361730084e-03,2.732680510738093e-12,-4.168297808583846e-12,2.070135585405087e-04,-1.165451230974185e-04,-1.273033985441989e-11,8.452815175754987e-05,-6.239258694851801e-13,-8.914523706342990e-13,2.002159211640195e-05,-1.028117443975647e-07,3.123961386409072e-12,-1.060182601817513e-12,5.867655332719823e-13,-1.857777266869575e-05,-8.374916860407662e-07,7.100019593075825e-13,-2.387475192126094e-13,2.159794466210917e-06,-1.014539644910477e-06,-4.357566918512755e-13,-3.125889716110082e-07,-9.690846846856755e-13,-4.434123931242497e-08,-1.818074215052026e-06,-3.222295356038299e-12,-5.959075362320894e-08,-2.064647506812980e-07,1.645071050642244e-01,8.671975242721680e-09,-2.155367726612036e-09,-5.076572587718406e-02,1.359766461584818e-02,1.312551725106756e-02,1.996670692159179e-09,1.219947777718965e-09,3.363759764773401e-03,2.136682362590619e-02,1.924078174962089e-02,-1.320526868189630e-03,2.304716833434542e-09,-5.822063964654605e-10,5.453181167609720e-03,-7.007398138360195e-04,4.946496040809187e-03,1.944801995097375e-10,-3.931037375808404e-10,5.080544426106228e-04,-3.264360025327466e-03,-1.533423040984577e-03,1.027249366317103e-10,-1.738252140642556e-10,-1.018711365640551e-03,-6.921093494277800e-04,4.990216318059458e-05,1.301073067409342e-10,7.425309670437779e-11,-3.642212164685388e-04,-1.814439733049768e-05,1.499788666986411e-10,-4.338454262897871e-11,4.891560737888405e-05,-8.553925718599379e-12,-2.188386765964406e-05,6.667899926425613e-07,-2.321633705890064e-12,-1.660011809639202e-12,-5.783297332019524e-06,-1.932610847663384e-06,1.711915241477057e-06,-1.463608401831114e-12,1.832573438443719e-13,-2.320623894599783e-06,1.480479804968723e-06,-3.936110235515193e-13,2.230982985927001e-13,-5.685911971494592e-08,1.652150853457504e-13,-1.310853278643372e-07,1.671762528182129e-12,3.452551427794927e-12,2.901696816218555e-08,-1.566188020906293e-12,-1.896261999745735e-12,-1.098465314011567e-02,-2.901761768498956e-09,2.057386933884171e-10,3.375057605038082e-03,1.596936187501509e-01,-1.127675423116588e-02,5.968224484562199e-10,1.493280553012835e-09,3.248613545997531e-02,-3.481455404779697e-03,-1.452565379942691e-03,4.758735910760052e-04,-1.327654327274511e-10,2.038691351386581e-10,-7.007408866091416e-04,1.433855980210084e-04,4.554048333852480e-04,3.416535967915919e-09,-4.353443409673575e-10,1.282758157016486e-04,-1.610572960749101e-03,2.669467485025353e-04,-2.407554302229495e-10,7.602553135654765e-11,-3.111200467754314e-04,8.029334502369108e-05,-1.860899491562141e-03,-1.916810848050488e-11,-8.085077149279621e-11,4.096027821216371e-04,-1.928554909904468e-04,1.041847039005094e-10,-4.031599914166004e-14,2.229047516760089e-05,1.057083470493494e-10,-1.779353689308307e-05,1.981457382336222e-06,3.805082540019109e-12,-9.996499730818846e-13,-3.558313827206369e-06,5.854550751182190e-07,-1.344484640459884e-06,-1.813331265387468e-12,-2.615617130886489e-12,4.216392907901053e-07,-1.700527075682272e-06,4.128741801038739e-13,-9.593705181991362e-13,2.729993970598848e-07,6.701233713400988e-14,-1.786908326081391e-06,-8.390723594836660e-13,-1.017594204496245e-12,3.335226680739781e-07,-1.204005539043386e-12,1.253719446697489e-13,-1.672457467875672e-02,-9.488805415693705e-10,-4.690969344221505e-09,-1.298186825845150e-01,-5.496085297838179e-03,-6.918784465695245e-02,-1.783984149238717e-09,-2.025827661595163e-09,2.467053441413784e-02,3.065239316554768e-01,5.110371125847741e-04,-1.657964869902452e-04,2.766602088862464e-10,-3.029039391591642e-11,4.946495588444223e-03,4.554004809723092e-04,5.621145639049914e-03,2.786121845524442e-10,1.156052449818441e-10,1.793522762144614e-02,3.547742616257471e-04,7.180655556285406e-04,1.967506542538950e-10,-2.033001775899773e-12,-2.670211259632623e-03,-4.397609329897336e-03,-1.260078110634559e-03,-5.376118041513829e-11,-5.932588244362906e-11,-5.258267345052088e-03,5.467306974637776e-05,1.753254884402282e-11,-3.145320756116940e-11,2.815063011645986e-04,6.844490836447047e-11,1.928513413622933e-06,5.711882055322947e-06,-4.504842030914970e-13,3.122480936716618e-13,-1.690956963439055e-05,-3.212924129803038e-05,-7.495144577933602e-06,-9.406881768088853e-13,9.551225438059984e-15,-3.529783561455810e-05,6.769487461962125e-07,-8.538918947766348e-13,2.753496376928140e-13,2.558962897460335e-06,-4.006206961862593e-13,2.773795806932236e-08,4.443895707498836e-12,-1.576934945400659e-12,1.744556713753725e-07,-3.149718888564664e-13,-4.321610237296248e-12,1.620222643093601e-09,-1.362925975393404e-02,-1.627400174146527e-01,-1.191514337469475e-09,2.461963024282113e-10,2.689390114337759e-09,-3.663063218085847e-04,1.276749069499978e-02,2.358857743025487e-10,-6.660013573385927e-10,2.009150680202416e-10,2.154915803530827e-10,2.188605121645987e-04,1.286639688523128e-02,4.563883438464088e-10,9.416746839759957e-10,-2.616994855848175e-10,7.941868751439287e-05,3.985333124067182e-04,6.861397962534127e-10,-4.588305666400618e-11,-9.430742549284972e-11,1.215886815002940e-04,-3.061227153978384e-03,-1.342577158192359e-10,-5.478567880495574e-11,3.767993493385236e-11,-9.099798456959572e-06,-1.575041828613866e-04,-2.511816839694903e-11,-2.242630254779931e-11,5.339430238997985e-05,2.545929701863775e-04,2.484121866839864e-11,4.094013740072603e-05,-1.781458039126163e-12,1.419846396041603e-13,7.996864203526948e-07,-2.122352186302158e-05,2.768449059031842e-12,3.663660162396309e-12,1.005415872724737e-12,-8.568716059561045e-08,-1.544699313826811e-06,2.197303891070761e-12,1.759886576590117e-15,4.161179871448414e-07,2.151447318712735e-06,-2.083424621323580e-13,3.686328690148360e-07,5.329454535544599e-12,1.320800416821078e-07,1.627835075396458e-06,3.386394079748732e-12,9.780717429641606e-08,1.939736907673072e-07,-4.453154163170639e-10,-1.555653378194544e-02,1.134168984938150e-02,-9.617394240325554e-11,2.137137165485561e-09,7.809553305451923e-09,-6.441758659923624e-04,1.572472736352714e-01,1.453058436277689e-09,4.974356157291843e-10,2.364116559798446e-10,8.691466266822795e-10,-1.536906896713906e-04,-4.300199092453532e-04,-7.451179869802827e-10,-4.594819323023075e-10,-1.628480065766742e-10,3.985343488635056e-04,1.260624909856428e-03,1.772662161097417e-09,-1.424031474185570e-10,-1.265780733338804e-10,-5.853946989132624e-06,-1.120392819162990e-03,-1.515803192204342e-10,7.829345894955822e-11,-6.052154113201324e-11,1.059700118258510e-04,5.906988932086136e-05,-4.720022283245519e-11,-5.975395523441646e-11,1.678997887159092e-04,-1.597934999191035e-05,3.605630508675529e-11,3.377637719899928e-04,-2.422322324104624e-12,-4.290806551429820e-13,1.783605828183841e-07,-1.447479720094839e-05,-3.967538557805860e-13,4.658310413851432e-12,-2.308734350908598e-12,8.853178340411226e-07,1.321615993534682e-06,-1.855766706195868e-12,-3.351657654728917e-13,1.381604285451338e-06,-1.059487820916916e-07,-3.027007232137944e-13,2.856554177637232e-06,-1.971103897790330e-12,2.151942041461169e-07,-1.242591030336222e-07,-9.436120883539641e-13,1.591367269224351e-06,-3.253701213966662e-08,1.736480136328732e-02,5.184875745546335e-10,2.038494074126300e-09,1.111931135701760e-02,-1.190304333310804e-02,-1.492766571248023e-01,-1.425124258462603e-09,-7.959683970531545e-09,-3.021544848843186e-03,-4.661211186766322e-02,7.919900475570387e-03,-8.854204483944338e-04,8.306108169569035e-10,-1.186486866378523e-09,5.080583678042410e-04,1.282756739930991e-04,1.793522785545010e-02,-8.900526083495898e-11,-2.092970956815070e-09,-5.132264630041340e-03,-2.195213766109885e-04,1.238480071076852e-03,8.480066115756729e-11,2.950690003965452e-11,2.502893950455661e-04,5.793286657319815e-04,-2.722614869806084e-03,9.150007326027078e-12,-5.239779072971361e-11,8.742338388327146e-04,1.969781017942122e-04,1.786150068036131e-11,-1.369095328558842e-11,-6.354513026848268e-05,1.156036919721889e-10,-1.610747491022394e-06,1.506134199040451e-05,-1.704634629033840e-13,1.756839680146765e-12,1.329656053788891e-06,4.748647844196601e-06,-1.883877869585738e-05,-1.456219094282547e-12,2.010107888694035e-12,5.040461635401641e-06,2.871551440582651e-06,-4.520657954407228e-13,-2.968031817485257e-13,-7.148709009119323e-07,4.670368085899180e-13,1.080801364086202e-07,3.304440998996192e-13,1.966904869535745e-13,-1.717202672982798e-08,1.992657689087805e-12,1.929870166032111e-12,1.329648401669852e-04,1.357294540536595e-11,4.262596060677638e-11,-8.182749860959364e-04,5.481892414834121e-05,1.080113160373130e-04,1.507878727219713e-10,-4.893639892815389e-11,-7.977634691217235e-05,5.129413766465048e-04,1.516950346503505e-03,3.448049531711874e-04,-2.233563099855864e-10,3.917925320304251e-11,-3.264359679691848e-03,-1.610573194392219e-03,3.547743550054486e-04,3.569924555814109e-11,-1.712617484279184e-10,-2.195214376897986e-04,-2.347598561481594e-04,1.776966869698276e-05,1.184419820042992e-10,-3.125897506392761e-11,7.667977334921347e-04,-1.037008415341734e-04,2.632335592628790e-05,5.118053840904664e-11,1.522724445011121e-11,3.324458864639980e-04,-2.905066250182378e-05,6.366535660545406e-12,-5.169528776596545e-11,-5.053148796739457e-05,1.456100074126743e-11,-5.851713851289545e-06,5.393534738281019e-07,2.184961904179683e-12,6.044408672473315e-14,1.914949097391249e-05,-2.236190995734953e-06,5.184475967695606e-07,1.411452121079506e-12,-1.194915621840162e-12,7.919907213972768e-06,-4.778098341603750e-07,-1.909373007623621e-13,-8.638092571311778e-13,-1.017835595394674e-06,-3.284991614166285e-12,-7.506507117876359e-08,4.809693795655084e-12,1.273285174260035e-12,-2.708939616472752e-07,6.049775485003873e-12,-1.380895750627385e-11,-1.741256468540540e-04,1.572239486595262e-10,-1.332989509904486e-11,-1.837663260423675e-04,2.215680333470543e-04,3.981350126334948e-05,-7.246892326037676e-11,-2.799786302763272e-11,1.327938251156383e-04,-1.684161314664026e-04,1.176076363224645e-03,-1.069674923502547e-04,-1.648155193875794e-10,-3.847581474114645e-11,-1.533422953352925e-03,2.669469021625862e-04,7.180657559462922e-04,1.800787598835655e-10,1.792181380942072e-11,1.238479930871590e-03,1.776955069845306e-05,-5.930003872263131e-05,-5.523398454462393e-11,-2.152113384893146e-11,-8.233038900572492e-05,1.898460229316690e-04,-6.901227804836793e-07,8.122421030216165e-11,-3.613599310240915e-11,-1.236031374744225e-05,3.255601509403925e-04,1.711484501651543e-11,1.504895904725382e-10,7.717665328415526e-04,6.925420777000439e-12,5.109299488424874e-07,-1.536764396610100e-06,8.506330723388110e-15,7.337430081824493e-13,-1.917678907755476e-06,4.503937799546378e-06,1.094703686998365e-07,1.598992614854853e-12,-3.810549381867259e-12,-3.837166395546289e-07,8.971821455506869e-06,1.345517883755967e-12,2.976189369666139e-12,1.975003989225565e-05,-3.817912826769849e-13,1.857030767932892e-08,-6.692385727844771e-13,5.519430679322407e-13,1.166011671791967e-07,-1.754780045042668e-12,2.091650161540854e-13,-8.987177811194897e-11,-6.449678546117317e-05,2.776251017187802e-04,-7.119410667477907e-11,-8.817501309573718e-11,-3.373615664099091e-11,3.128516181324076e-05,-7.386159021908091e-05,-1.099313564008716e-11,1.492815476713784e-10,2.830544378681082e-10,1.547299176433700e-10,-7.795932797864173e-05,1.921337327134650e-03,1.080967475319100e-10,-1.004235044457156e-10,8.110929796656006e-11,1.215887095416479e-04,-5.854073363687651e-06,9.593087053633538e-11,-1.556261963369108e-10,-2.314556640847024e-10,-2.025419086211919e-04,2.136994401097196e-05,4.728770927650489e-10,-3.966412873544827e-10,8.191316253003442e-11,-2.204723905363713e-06,-2.034735247083970e-05,2.040668389471449e-10,-3.889910170336169e-11,1.294072948508151e-03,-8.175478142948131e-05,-4.603496555332211e-10,2.666832904379177e-04,-2.213293225103610e-12,-4.656955619744234e-12,-4.918576634468133e-06,3.628026293552214e-07,6.357838730731625e-12,-6.066897532982497e-12,-9.876137920538632e-14,-6.481650734834382e-08,-3.491707680913767e-07,1.993265895585554e-12,-1.904135520111200e-12,3.355129138589789e-05,-2.133169049787359e-06,-5.983119739022566e-12,7.624117900087560e-06,8.137307086656338e-13,1.106957729013172e-07,4.556260348255744e-08,-3.716776790822023e-13,1.977857363284039e-10,-2.969155641734151e-07,-2.020956626407690e-11,2.086741831398828e-05,-4.311711104691144e-06,1.239321856340365e-10,1.216252972925013e-10,9.106440122716924e-11,-5.349506084268407e-04,-1.292678322015690e-05,-2.882642162971979e-11,-1.069043184733167e-10,-8.860721771198614e-11,1.087675965564558e-10,1.878693975213804e-03,-1.998424830946603e-05,6.192337632736762e-11,3.247586816887108e-11,4.518443678876766e-11,-3.061226682287674e-03,-1.120392616405668e-03,-1.951609086985750e-10,-1.341828180344817e-11,-4.592098177356364e-11,2.136979605709111e-05,7.396398312839658e-06,4.371230584182822e-11,-2.259378673823209e-11,-2.537179646560165e-12,4.929723747251600e-04,-6.684996988991202e-05,7.960738282069969e-11,-1.307793063884666e-11,4.013234473241240e-05,-8.053327801940742e-04,-5.181638084876440e-11,-1.317839773811392e-04,4.637245789550817e-13,-8.207844856741239e-13,3.840380455621180e-07,1.883457997204393e-07,2.664674127255307e-12,-7.982370458861890e-13,-7.198636242167743e-13,1.274397402572849e-05,-1.509823603715012e-06,2.238539722634612e-12,-1.149758583671549e-12,1.037456433612349e-06,-2.068304172837137e-05,-3.173546593672849e-12,-2.809316682836430e-06,6.036451125299016e-14,-8.448185158300672e-08,-2.543004979782300e-08,4.908657645179390e-13,-1.567625483828467e-07,-3.269902349702571e-07,7.537780372693716e-04,1.540016568190775e-10,-1.062707204401898e-10,-1.601367450912348e-04,-4.574997705188285e-05,-1.047302043554748e-04,8.771235436545509e-11,2.635491819609925e-10,-1.374914058125139e-05,3.911465908098884e-05,-6.332132154290414e-03,-1.469033920717004e-03,-1.432141145153829e-10,-2.005535142031841e-10,-1.018711204700511e-03,-3.111200594157881e-04,-2.670211473583325e-03,-1.038507469051230e-10,-1.093923237025944e-10,2.502894959414006e-04,7.667979052614120e-04,-8.233062342499283e-05,-5.944497148836027e-10,2.465024974056017e-10,1.360538921027098e-04,-1.691366583863306e-05,1.605263654317720e-04,-1.372698893861942e-10,-1.983300708800262e-10,1.652849020657032e-05,9.066379967987655e-05,8.148169743899229e-11,3.487603901228574e-10,-1.502397278904657e-05,-9.012538040277284e-11,2.024502227013132e-05,-1.932310842812819e-06,-8.046821780625834e-12,3.504716089242359e-12,3.700810012265550e-06,-3.949992508108096e-07,4.688633990243889e-06,-2.384718567619694e-12,-2.759286497094273e-12,3.365481551099996e-07,1.957796766395146e-06,2.514036954950377e-12,5.349741378213249e-12,-3.317179557559721e-07,-1.064731510738096e-12,1.864122429463296e-07,7.224651147717061e-13,-6.405606948280916e-13,-4.461076159736676e-08,8.100785102651870e-13,-2.126313556700079e-12,4.613910467659681e-04,-3.031382425966499e-10,-1.332111979297154e-10,-6.797615100355284e-05,1.239335779413670e-04,-4.075229868636337e-04,4.249579944415218e-10,-2.305940217875390e-10,2.077182710239807e-05,-5.926254673906470e-05,-5.191915251986491e-03,5.341388710143638e-04,1.171547673891636e-10,-5.095254830655312e-11,-6.921094702594236e-04,8.029344367352350e-05,-4.397609432922448e-03,-2.961319687076754e-11,8.746208258561989e-11,5.793285411433393e-04,-1.037001721515542e-04,1.898457189134919e-04,-3.456389962003871e-11,-1.167508612684316e-10,-1.691344616192862e-05,3.482558787357930e-05,-5.448989597430577e-05,-6.786114093319714e-10,1.248436232336260e-10,4.013669232102498e-06,-1.577099211346064e-03,3.343192657255928e-11,-2.580962003605806e-10,2.090547559163821e-04,-1.695893006782908e-10,-2.360646588729067e-06,4.782828588864058e-06,7.547643720175923e-14,-2.661702857582366e-12,-4.049127764635867e-07,8.920144930066938e-07,-1.146126604813905e-06,-8.955543886415882e-12,1.223485174290015e-12,7.360352248783657e-08,-4.226615473250525e-05,-8.909802695420948e-13,-4.254406390682868e-12,5.838892169516339e-06,-3.689455491821484e-12,-1.420310395186105e-07,3.115523574359619e-13,9.637784550033843e-14,2.522886889576980e-08,1.096466171540767e-13,1.980961530716013e-13,1.135565727139692e-04,6.320822429146330e-11,1.313078094424630e-11,-1.548028195190418e-04,3.021693549893482e-05,9.416825884229331e-05,4.280690516893657e-11,9.709515876674680e-11,-5.153333461147285e-05,9.158425926211017e-04,-4.965529173979789e-04,7.416406135352106e-04,-1.284555012322136e-10,-6.665326110353927e-12,4.990204733623793e-05,-1.860899435407811e-03,-1.260078061457618e-03,-6.396631860293830e-11,-1.305537297318765e-10,-2.722614607024049e-03,2.632333381433604e-05,-6.902833845567848e-07,2.435214533768773e-11,7.849512539355272e-11,1.605264389627988e-04,-5.448905187531887e-05,2.100327818742452e-04,-5.001391180298064e-11,1.071922366231881e-10,6.936236196201158e-04,6.220181042307076e-05,1.516579132793016e-11,7.552969990673748e-11,1.240700700936733e-04,-4.164118135166234e-11,7.913782828863992e-07,7.029757520094832e-08,-5.391817253664594e-14,4.614069844998579e-13,3.783402583503535e-06,-1.082849862828506e-06,5.314022651206699e-06,-1.191230777975407e-12,2.083576122378198e-12,1.750618256561091e-05,1.296601039196704e-06,2.208774013724580e-12,6.669066001387654e-13,2.888759253520336e-06,4.424352422974140e-12,-1.963399056626082e-08,5.152666524827775e-12,-1.874813870588358e-13,-1.572354712073244e-07,3.383403110111382e-12,-1.303028859394946e-11,6.501642391400752e-11,3.402638944374735e-05,-5.438104574881409e-04,2.365043894937700e-10,8.033825484324753e-11,1.330506131529984e-10,-4.379775498667067e-06,-4.042571687426552e-05,-3.836277401174652e-11,-2.610950973503038e-10,-7.337006062825158e-11,-1.941760745048503e-10,-1.521709133696806e-04,-2.927098126528343e-03,1.186745410563001e-10,-7.307460994580559e-11,9.192573454569435e-11,-9.099815904197586e-06,1.059699987916998e-04,-9.362579008909271e-12,5.186847891928444e-11,1.254881291399037e-10,-2.204428944430048e-06,4.929723672828172e-04,-3.437406067916208e-10,-7.285287750035868e-10,1.392896745340334e-11,4.775958122899600e-06,1.801055549000815e-05,-3.443243653989643e-10,3.102373184044663e-11,9.369958665387491e-05,8.169819181527444e-04,3.297669123732311e-11,8.284091547195455e-05,-4.644760110059882e-12,3.019583615124365e-12,-7.874583524959941e-08,1.274774047458809e-05,-6.073416623099419e-12,-1.092717532600377e-11,-3.287286796435775e-12,1.192803528587549e-07,4.457396232575830e-07,-4.333721148800852e-12,5.190640123399521e-12,2.629904046911525e-06,2.067860772949876e-05,2.102267662078797e-12,2.334624124657410e-06,-9.619271261148907e-13,-9.104084993020449e-09,2.974808299804649e-08,8.736256689058839e-14,-4.078545127023544e-09,2.997111215751296e-07,-9.464304903577893e-11,-8.272130682652465e-05,-2.391063936623031e-04,2.073842322768108e-10,7.054794800450296e-12,2.852588172517620e-11,-3.337004770089875e-05,1.763441406183211e-04,6.562761853713254e-11,-1.408834722348789e-10,1.465003000857232e-10,6.793690720938173e-11,7.536088283854373e-06,-1.788352174729262e-03,1.980848847394414e-10,2.971222542697190e-11,2.425483922077534e-10,-1.575042399149603e-04,5.907000593948378e-05,2.012906373527007e-11,-4.566431462943051e-11,-3.154197155272457e-10,-2.034648762989604e-05,-6.684980717631759e-05,-1.198549466198788e-11,3.328400348620968e-10,-2.777274947041062e-11,1.801035332098571e-05,1.779477021436699e-04,-3.956804746424312e-10,-9.124045623380598e-11,5.502987570838612e-04,-8.502591130722352e-05,-4.802008479380369e-10,1.159676104810048e-03,2.689888268744456e-13,-2.936307263135571e-12,-5.951981280648858e-07,-1.550862232459766e-06,6.613951300828172e-14,5.257347880486693e-12,1.672073474827895e-12,4.569237002173955e-07,4.347952828818574e-06,-5.752742569395557e-12,-1.438201926520970e-13,1.467863242229509e-05,-2.254629595619204e-06,-5.963563533118800e-12,3.033494931178094e-05,-6.054799898608106e-13,2.954978095842214e-08,-2.172384571287072e-08,4.257481854765622e-13,1.042114226067209e-07,-4.257348461587013e-09,3.062861870958949e-04,2.643243072930700e-10,-1.503063439814392e-10,-1.834292896456033e-05,1.110761797825261e-05,-7.394981848581385e-04,-5.904661939314722e-12,2.728852059231318e-10,5.860141419343623e-06,-1.685144879863118e-04,-3.148494579379586e-03,2.111283810515602e-03,-1.172479883551317e-10,7.834990894673790e-11,-3.642212952279880e-04,4.096028327553416e-04,-5.258267040200669e-03,-1.638452696183087e-11,9.079967097656110e-11,8.742335359412587e-04,3.324461326873759e-04,-1.236058769598575e-05,1.049540090942066e-10,3.491284571717640e-10,1.652759289830596e-05,4.013744271625084e-06,6.936235119566137e-04,-9.433392770196896e-12,4.803679723897718e-10,-1.236780493065180e-04,2.558540667333139e-04,2.137484076900825e-10,4.680884778289401e-10,-3.255963326695639e-05,-1.586985829841902e-10,9.099769815884931e-06,-4.479983388472334e-07,1.501382824318482e-12,4.230926778683698e-12,5.281235261092813e-07,6.321228967073719e-08,1.853054339693908e-05,1.088856053509868e-12,6.243000918347756e-12,-3.412332777952928e-06,5.824045707909959e-06,4.679861570454901e-12,4.248491603114315e-12,-7.623679734776032e-07,-5.425267209623357e-13,-4.726556459989710e-10,-6.538106419692168e-13,6.379484948107960e-13,1.663675425944390e-08,-4.481771409700705e-13,1.733506752129023e-12,8.312714010049167e-04,-2.469643407514625e-11,2.926792156086028e-11,1.801543771450686e-03,-8.513272257029521e-04,2.357490425675785e-05,1.190394253253425e-10,-7.714995954872052e-11,2.891607538899169e-03,4.979731957952589e-04,5.992839083971397e-05,3.753492618071687e-05,3.672258663287575e-11,2.431631886564433e-12,-1.814459311179392e-05,-1.928555708366713e-04,5.467322611379507e-05,1.993461588269261e-11,-5.624251241904918e-11,1.969780593640675e-04,-2.905069793298629e-05,3.255601393476130e-04,-6.698898437386889e-11,1.452174722516538e-11,9.066387096829532e-05,-1.577099239355994e-03,6.220179260298495e-05,-3.281331861703864e-11,-1.104123757006645e-10,2.558541667166281e-04,3.216061728884524e-05,2.890743923375451e-11,1.816838358175380e-11,1.141520041693242e-04,-5.230934317066167e-12,-5.151689024719099e-07,8.523621025561670e-06,-4.953276661429110e-13,7.701276066940064e-13,2.186378884049253e-06,-3.864617075873174e-05,1.268850362031260e-06,-1.997945182632705e-12,-1.255228597198338e-12,5.589998971278047e-06,7.665430543742505e-07,5.618162778461361e-12,9.215088679940075e-13,2.486216378342564e-06,4.411259040727698e-12,-1.517498969470740e-07,1.406288363494928e-11,-1.062794848046623e-12,-6.507035732334190e-07,1.309745785923066e-11,1.073158428244579e-11,-2.769801270192874e-10,-3.067772333857533e-03,-1.779798299644424e-03,-2.315530921618009e-10,-4.090487525478650e-11,-3.000421140878222e-10,-2.737598016774142e-04,-3.736577918521356e-05,2.718721479629494e-11,1.067208930396126e-11,-2.592753011965575e-11,-1.223547303085284e-10,3.712926332100000e-04,2.070136493939863e-04,-6.581857146543122e-12,2.091882034950377e-10,1.652311160715854e-11,5.339426502847915e-05,1.678997316436433e-04,-8.073192881290747e-12,8.861360524215704e-13,1.183299205985697e-10,1.294072910455065e-03,4.013233981023000e-05,-1.232881748051786e-10,-7.148660536782468e-10,4.395191353615007e-13,9.369964606471396e-05,5.502986850921952e-04,-9.284347502933894e-11,2.132487184504145e-11,1.306608218615131e-04,-5.400046401781264e-07,-1.769986761501501e-12,-1.207133649051449e-05,-2.362773702032019e-12,1.245413098333196e-12,3.184681809073938e-05,9.842982706166583e-07,-8.068781824976071e-13,-8.745310332929444e-12,1.258267963823348e-12,2.497821196293407e-06,1.283930462376661e-05,-2.682523834113443e-12,-5.239249403966015e-12,2.835846849938419e-06,-1.631839511364241e-07,9.001968755140597e-13,-1.568532378788354e-07,-1.478046569426994e-11,-4.999414400780586e-07,-3.901799041975962e-07,2.311196078214053e-12,1.724620579145855e-07,-2.303121416970018e-08,-1.792994948292934e-10,-1.737154652620528e-03,2.671919182180667e-04,3.668458413437234e-10,1.110424473544958e-10,7.229412420013524e-11,1.339013825351006e-05,1.370777026207048e-03,-4.577520508412784e-12,-3.919349747240913e-10,2.955970897929376e-11,3.389900439854384e-11,-3.928097724730810e-05,-1.165451654563728e-04,-3.408834971660170e-11,-3.492316610295728e-12,8.389167075703565e-11,2.545928760412561e-04,-1.597931705127838e-05,5.028741751586155e-11,-4.077657774628967e-11,-5.325617669675743e-11,-8.175494733320743e-05,-8.053327894909564e-04,2.952319289092397e-10,2.405220822074777e-10,8.278300869789363e-11,8.169819136581708e-04,-8.502601755826229e-05,3.819664650970937e-10,-1.432298538929994e-12,-5.400085862294429e-07,8.641905156229142e-06,7.609453173401588e-11,1.016816252581020e-05,1.364656583576429e-12,-3.206644019318128e-12,-2.042876546867405e-06,-2.069804070428053e-05,4.312161912632191e-12,3.490413793086286e-12,3.950914407015747e-12,2.067490276663644e-05,-2.130373487379735e-06,2.236006651777096e-12,-3.094714971091853e-12,-1.304931899069144e-07,2.361410889754526e-07,4.722827170666299e-13,9.838205611048795e-08,1.583600055317741e-12,-9.326375339858820e-08,3.569322132863470e-08,1.791892268377361e-12,-2.398122960642181e-07,-2.660328883690963e-08,1.844187017168773e-03,1.680423395569505e-10,-2.944364545337967e-11,-3.809890169543423e-04,-2.757479042045530e-03,-1.165932556410131e-03,-1.910173920629111e-10,-8.626585701602917e-11,-6.145106199663109e-04,-1.697261790417011e-04,2.251581627241334e-04,7.631034017304299e-05,7.268373989596237e-11,3.929440635024293e-11,4.891575310960791e-05,2.229045846716316e-05,2.815063147841811e-04,5.135969524055949e-11,-7.314569873760909e-11,-6.354508946783597e-05,-5.053155095003776e-05,7.717665335840657e-04,-3.145439470208839e-10,1.002433416905042e-10,-1.502387993957039e-05,2.090546829580880e-04,1.240702159722466e-04,-2.672816625908017e-11,-2.832032050850024e-10,-3.255947748205164e-05,1.141519599800550e-04,5.828705432585208e-12,4.206444743156220e-11,-2.792958012769158e-05,7.967904943616348e-12,-1.104580994049949e-06,1.978120761865835e-05,-3.178585423792331e-12,3.184815810519557e-12,-3.607316073096120e-07,5.505369329391551e-06,2.840272958828081e-06,4.787703773837480e-13,-2.081521863212315e-12,-7.451969401978342e-07,2.692681377507457e-06,1.809335863587967e-12,-1.018265260927932e-12,-6.776024940061345e-07,1.481427942904208e-13,-3.237134504222591e-07,-1.327214084011538e-12,2.115111726832182e-12,1.021965159880885e-07,-1.560598692103557e-12,-1.559956043914677e-12,-1.750848421717086e-10,1.714937618527380e-04,-2.038736711403893e-03,1.093854586806089e-10,-8.005388416157145e-12,-6.803447553832337e-11,-2.634744405468745e-04,-2.535652346384747e-03,3.678825698533299e-11,-3.384881371328053e-10,-1.553938976555615e-11,-2.447213867409895e-12,8.669324495449742e-05,8.452828167986363e-05,3.362938784339952e-11,1.983732380846513e-10,-4.748055335122611e-11,4.094017902264545e-05,3.377637575678269e-04,1.059681650464776e-11,-2.658563127632911e-11,-1.031886337711860e-10,2.666832830244772e-04,-1.317839165892684e-04,1.391003376967448e-10,7.672552875416291e-10,1.000302745942922e-10,8.284089018177095e-05,1.159676181011277e-03,1.109894715328597e-10,2.153585039255123e-11,-1.207131703373954e-05,1.016814018607148e-05,-4.416017654835368e-11,-1.197711243092424e-04,1.632753920134866e-12,-1.210961252907784e-12,6.211469079673355e-06,-2.875693953611954e-06,1.115417433868945e-12,9.685183657335547e-12,-6.874351920265512e-13,2.236817993009642e-06,2.874357949098592e-05,3.840010209045113e-12,-4.686815203768224e-12,-3.508279282676529e-07,1.309190610498663e-07,2.069121908140042e-13,-2.639861238856032e-06,-1.298671516845796e-11,-1.272494016563262e-07,-2.355548914342179e-07,1.426598785818956e-12,4.166147409107490e-07,-1.712510177072424e-08,2.995562216809359e-06,7.875103245293316e-13,-1.226962863838251e-13,8.569184007025872e-08,5.533716242615879e-07,1.492679956095687e-06,-4.496574626119473e-13,-5.247620734643620e-14,-1.157525056618422e-06,3.938872656277903e-07,8.809946052153756e-06,4.122609252416426e-06,9.735357310320117e-13,4.019204112423285e-13,-2.188386626299823e-05,-1.779353730436938e-05,1.928500526334938e-06,-4.986747557069291e-13,3.043961030371301e-12,-1.610743225459594e-06,-5.851709225586949e-06,5.109284153435318e-07,4.551010625436293e-12,9.140100140540507e-13,2.024501996008631e-05,-2.360656625423369e-06,7.913733551030869e-07,-3.503425710783647e-12,-3.367693899644710e-12,9.099769272086587e-06,-5.151685783863965e-07,-3.351488966443386e-12,-9.851070978187291e-13,-1.104577661847909e-06,3.116052361624838e-12,-1.930373896887127e-06,1.809367644834724e-07,1.033790729316657e-12,-5.041694783353608e-14,6.508044275768845e-06,-7.425801602813191e-07,2.181452397865179e-07,-3.974647577197145e-13,4.568238290488905e-13,2.816035155657046e-06,-1.807913802127081e-07,1.727133709543929e-13,2.539488819080337e-13,-3.811896993486020e-07,3.618948511944192e-14,-2.659960034663979e-08,1.207614391829984e-12,-7.599889785282256e-13,-1.007002149993312e-07,-3.392937232819642e-12,-6.008072083915910e-13,-4.934109714522533e-07,-2.372294831154062e-13,-7.106534427062797e-14,1.156205438100785e-06,2.816293014864973e-06,5.757649912481124e-07,7.999554090501542e-13,-5.893547466554128e-13,-2.103871196245197e-06,-2.978182727626725e-06,3.880672549576310e-06,-8.009809988152972e-07,2.816512933315558e-12,-8.535949484766513e-14,6.667873664341430e-07,1.981455051835753e-06,5.711883564073180e-06,-2.935477675770276e-13,1.851361468776425e-12,1.506134190611408e-05,5.393525031604991e-07,-1.536764019397040e-06,-3.253335295018393e-12,3.964003367675619e-13,-1.932309250670599e-06,4.782831902162506e-06,7.030012950543130e-08,7.592014072467846e-13,8.145305083562363e-13,-4.479933480060740e-07,8.523621372488290e-06,1.920221511672845e-12,-5.945834076304336e-13,1.978120753859571e-05,5.216073104585221e-13,1.809379603685360e-07,-5.037630334687605e-07,1.077433214230557e-12,5.371527672465023e-13,-6.307709821405025e-07,1.517723539350322e-06,3.348554770909835e-08,4.979016906069748e-13,5.473001324259869e-13,-1.505923446149526e-07,2.861597555233185e-06,-3.730069438687535e-14,3.964301921554036e-13,6.460322901034210e-06,4.050593197403918e-14,-7.415457804159821e-10,-2.044106848588130e-12,1.871255078135326e-12,3.926068383969814e-08,-1.752672468690265e-12,3.377030182311532e-12,-7.574351784124041e-13,2.800764711627767e-06,2.666622346916112e-06,3.741999727634005e-13,-3.141324947162221e-12,1.178751766417535e-12,1.998101107663940e-07,-2.233684206196286e-07,-1.040938597921855e-12,4.045135994151945e-13,9.725865252184940e-13,-2.011704777257360e-12,-1.496189387697908e-06,2.002159481176547e-05,-2.205535412493190e-12,-9.583683733750704e-14,2.951066847791853e-12,7.996877262868974e-07,1.783617397317605e-07,-2.749835653691682e-12,-2.056954050181630e-12,-4.906072361909487e-13,-4.918579530392969e-06,3.840424541724118e-07,7.928536712049311e-12,-5.767784126601701e-12,2.032984527543484e-12,-7.874950727173382e-08,-5.952108494749659e-07,4.582224493626713e-12,-1.772431923884090e-12,3.184681962085218e-05,-2.042877233727419e-06,-3.141950139415748e-12,6.211467409264264e-06,-1.106338502941637e-12,-1.370643080068763e-12,-1.616129396331210e-06,1.118071910417833e-07,4.488904379281345e-12,-1.746210354772568e-12,-5.935338629648145e-14,-2.329776020683746e-08,-1.569095534072059e-07,1.519228574868027e-12,-3.526340536219836e-13,1.074198146216346e-05,-6.854920950897852e-07,-1.021220801694957e-12,2.265500754883010e-06,-1.131734484276432e-12,3.215325207546527e-08,1.556553684981975e-08,3.513816379337529e-12,-3.637738740512725e-09,-8.540281301122190e-08,1.387747058606195e-12,2.889398408444565e-08,1.219727789638814e-08,2.756865454998313e-12,-2.645061998557193e-13,-9.369308720269061e-13,-1.093080424157897e-06,-1.471021436774308e-07,-1.310836725494737e-13,3.680227588712857e-12,-1.149595812841688e-13,8.181775753100092e-13,1.072546076849867e-06,-1.028121259540537e-07,2.200113982481274e-13,-2.157084165893525e-12,3.300501915103424e-13,-2.122352092533594e-05,-1.447479908776700e-05,1.004534745155014e-12,5.598336721438920e-13,-7.492755753641751e-13,3.628009405260422e-07,1.883459563084048e-07,1.437232680849504e-12,-1.459895943327564e-12,-5.504535650261883e-13,1.274774017442420e-05,-1.550865088512068e-06,1.872731964135663e-12,-1.350046165348709e-13,9.843034164868050e-07,-2.069804361151137e-05,-4.735023448656572e-14,-2.875692848103188e-06,8.682247756462952e-14,-5.806574373250133e-13,1.118059279207271e-07,6.107196018755310e-08,-5.625403868410278e-13,2.931676184260347e-13,4.470839480088294e-13,4.213277090012830e-06,-5.000467581276042e-07,-6.333141860145385e-13,-7.717529533555282e-13,3.742111144776218e-07,-6.764337309166141e-06,9.674489169109745e-15,-9.690000822403937e-07,4.889838724802868e-12,-4.781961947480477e-08,-1.639015923762406e-09,1.734085144452399e-12,-5.313420277995926e-08,-1.202784125665127e-07,2.715078659144626e-06,-1.198889210303922e-12,-3.338398844647179e-12,-1.633850666616607e-07,-5.120301249337646e-07,-9.816007792070657e-07,-4.436038243562171e-13,8.177782778883298e-13,-1.766658674622572e-07,-6.198639756865714e-08,-4.119854684392119e-05,-1.711628277447589e-05,4.085546680669636e-12,-1.712869353999940e-12,-5.783293690935467e-06,-3.558314083006412e-06,-1.690956731397602e-05,6.682924988157400e-13,8.772928308850607e-13,1.329657211247501e-06,1.914949256729512e-05,-1.917683069475848e-06,-5.554365142367940e-12,3.599228782149611e-12,3.700809042716039e-06,-4.049166390156190e-07,3.783401248762125e-06,-2.639650377272899e-12,-1.699341827058847e-12,5.281319560175553e-07,2.186375737724844e-06,-6.935947171410129e-13,3.666467219398109e-12,-3.607304557016166e-07,8.769553450288663e-13,6.508045167218187e-06,-6.307727927232763e-07,-5.131449152906639e-12,1.298580258701006e-12,1.221511657317528e-06,-1.301511978124695e-07,1.397241548410822e-06,-4.575742967268945e-13,-1.145371169309350e-12,1.433213959502204e-07,7.554007124029395e-07,-4.519612695936646e-14,1.207241285517259e-12,-1.261030578557918e-07,-9.402725126319971e-13,5.288096361084226e-08,-1.941398741114178e-12,3.124324153227092e-12,-1.501170033945719e-08,1.643600634658735e-12,1.052333817025649e-12,1.510418340777533e-06,4.160068897362014e-12,3.499098636099063e-13,-4.000663130692646e-08,-2.162904036498885e-06,-3.203489647241715e-06,-1.059310224922235e-12,-3.289461370152000e-12,-4.390471138684973e-07,-6.291740645938430e-07,-1.975009632321592e-05,3.944592919006275e-06,-3.396776436608869e-12,-5.623142215082386e-13,-1.932609554642891e-06,5.854546736797014e-07,-3.212924246149839e-05,3.146818288894405e-12,2.910910097182303e-12,4.748647723394227e-06,-2.236183182831193e-06,4.503935641425593e-06,-1.538461737558352e-12,-1.359118306545251e-12,-3.949970512345068e-07,8.920101013234786e-07,-1.082859941766169e-06,-7.068940325720651e-12,5.644425008643436e-13,6.321226237763935e-08,-3.864617020366835e-05,9.271360073773306e-13,-3.013735792761153e-12,5.505370095011603e-06,-2.137655987213877e-12,-7.425770367219864e-07,1.517722346487389e-06,-6.585453203301594e-13,-6.330549217188211e-13,-1.301507275294679e-07,2.909700446531054e-07,-3.494559563320111e-07,-3.666715257796494e-12,-3.668869234860962e-13,1.948525143225139e-08,-1.322465964574319e-05,1.187546997467752e-12,-1.637279898931074e-12,1.851676799236110e-06,-1.650714842028833e-12,-4.089987379463643e-08,-1.191427827977536e-12,6.511844261228719e-12,7.690961618260260e-09,1.510648304838583e-12,6.373398312695150e-13,1.323614774140927e-06,-5.857183345841812e-13,9.816172086682434e-13,1.404680709428649e-06,4.453287729465252e-07,2.349123402152319e-06,1.240774593857324e-12,-4.420340601766322e-13,-1.004078806419668e-06,1.758972039145641e-06,-3.285023318692407e-06,1.686343810490592e-06,-1.237929020646443e-12,-6.263413131739350e-13,1.711914577486427e-06,-1.344483772447859e-06,-7.495155739101569e-06,1.769632220974461e-13,-3.751830397834366e-12,-1.883877520934399e-05,5.184532443209813e-07,1.094662221700002e-07,3.814944409262346e-12,9.269799468339399e-13,4.688636331148679e-06,-1.146115701163241e-06,5.314019391369004e-06,-8.999524757529746e-13,-1.379965024119017e-12,1.853054599618215e-05,1.268851176356664e-06,-1.815274677217141e-13,-1.524583099510350e-12,2.840274557194593e-06,1.140683641134066e-12,2.181464619506635e-07,3.348558456203384e-08,-7.400493464511997e-14,1.325537711341771e-13,1.397242794061646e-06,-3.494523584175161e-07,1.756102432998241e-06,1.287195926704926e-13,5.313184041495740e-14,5.958201662266756e-06,4.413607097774950e-07,6.571597583928932e-14,-1.114836944526323e-13,9.815432056599716e-07,3.594323342067429e-13,-1.168259532958692e-08,-3.187724027608693e-12,8.630934103847034e-13,-7.573857879503844e-08,2.124321635599242e-12,-1.863056743255790e-12,-9.177511503265009e-13,6.454973831084495e-07,-1.152117765367589e-06,-4.934038917786651e-13,4.507800194177576e-13,-1.059757792236354e-12,-3.208223326292639e-08,-5.019728218246868e-07,6.546599506307044e-13,-6.828939095703892e-14,2.700897041547540e-13,-1.835920358186863e-12,-1.496011359437096e-06,-1.857777101391361e-05,-1.228792306132979e-12,-1.474988393617610e-12,-2.934926032379039e-14,-8.568735268109936e-08,8.853184693266758e-07,-1.451070892848994e-12,1.387306374003219e-12,1.748393972249652e-12,-6.481237962888955e-08,1.274397272229736e-05,-5.418542682398642e-12,-1.146146456465007e-11,-2.577163332067267e-12,1.192802412662041e-07,4.569256989588543e-07,-4.226997178820348e-12,2.098078419801442e-13,2.497820740328779e-06,2.067490334471697e-05,-2.378346517040454e-12,2.236820100668256e-06,5.875746667278791e-13,2.050701276233471e-13,-2.329527609675750e-08,4.213277741068632e-06,-1.845491841414948e-12,-1.468323732905015e-12,-2.336060700959699e-13,3.857260953492532e-08,1.485776861199381e-07,-8.972004145216859e-13,1.378584457185196e-13,8.406993129246357e-07,6.799595046322976e-06,9.240102784470514e-14,7.495547514239933e-07,3.746405520466458e-14,-7.164494966093042e-09,4.785621936296405e-09,-9.247817242394434e-13,-5.709219872928084e-09,1.127133073212761e-07,-4.499262273935321e-13,-2.824087322773293e-07,-2.253078061730596e-07,-1.011086839967788e-12,-3.654102893730933e-12,1.176365814972709e-12,-2.829604196607004e-07,-1.484077889595452e-06,-1.076068456386500e-12,2.753862336588678e-14,1.543486954683415e-12,-2.654289065043951e-12,-3.543201792991353e-07,-8.374918087574232e-07,1.655320189993021e-12,-1.093610068651375e-12,3.971962197826543e-12,-1.544699164274472e-06,1.321612258854689e-06,9.747796479703508e-13,-1.059940644171915e-12,-5.245354408398096e-12,-3.491631117445519e-07,-1.509822459545606e-06,9.579774563174975e-13,3.588826281612046e-12,-1.379232342454751e-13,4.457374710951212e-07,4.347950810717741e-06,-2.545219574762903e-12,6.020084576171627e-13,1.283930416374374e-05,-2.130371219126741e-06,-4.679299612392849e-12,2.874358129367784e-05,-3.115330106415089e-13,-2.247719932206925e-12,-1.569070926891985e-07,-5.000467300873747e-07,8.238413245671907e-13,2.134489067422254e-12,1.274345923454135e-13,1.485784943282062e-07,1.430682416863922e-06,-2.911460751102278e-12,-1.391750976763952e-13,4.534095432397894e-06,-7.217619887753316e-07,-2.021193583355296e-12,9.719096641402756e-06,2.505232813438625e-12,8.423488527086323e-09,-1.231765312870465e-08,-5.435163668779230e-14,3.364693877764636e-08,3.428515085154777e-08,2.274393411203243e-06,-1.519317386996191e-12,3.674360388308167e-12,-3.770607356173602e-07,5.021027758148790e-07,-3.243055264300591e-06,5.651871220887180e-13,-9.958078219068540e-13,1.581648366483599e-07,-4.301789404461514e-07,-2.299289278049237e-05,3.100096258940281e-06,1.009884910948198e-12,-1.749600659757402e-12,-2.320622831696415e-06,4.216383543255897e-07,-3.529782908274308e-05,2.579042816030904e-13,-3.326456602680316e-12,5.040457115202817e-06,7.919909588978896e-06,-3.837200090370819e-07,4.093939418647972e-12,3.228241136383956e-12,3.365353260775678e-07,7.360494258142463e-08,1.750618111097001e-05,1.113840523160422e-12,8.953559784631786e-12,-3.412337064711712e-06,5.590000262738853e-06,3.559189204830050e-12,4.440058051747516e-12,-7.452001513377678e-07,-3.298734900009012e-12,2.816035596786687e-06,-1.505936715558139e-07,-6.277310283416782e-13,1.771965693676634e-12,1.433187634053494e-07,1.948518855461308e-08,5.958201642058205e-06,7.749085354331886e-14,2.642856800914242e-12,-1.127660796406369e-06,1.950543002104172e-06,1.201582336662205e-12,1.291854350728915e-12,-2.600790596901845e-07,-6.342548679048758e-13,-2.260399039470971e-08,-1.733483568796718e-12,6.788097062935390e-14,9.668616120955071e-09,2.917574130140872e-12,-3.688001569564865e-13,-2.419377663899756e-07,4.822971070531402e-14,-7.163482066323406e-13,-1.738778584677859e-05,-5.213522988071476e-06,-4.025540880808897e-06,9.710883685409631e-13,-1.880530794885989e-13,2.132468778192001e-05,2.819165798557320e-05,3.184220370795848e-09,3.589563509462430e-07,2.076082226417357e-13,1.523370641866112e-13,1.480479008105083e-06,-1.700527403363935e-06,6.769477379041029e-07,-2.055471547703153e-13,-3.834300094669588e-13,2.871551897459496e-06,-4.778090934100409e-07,8.971816311495919e-06,2.724481422971900e-12,2.583799755286810e-12,1.957797919461188e-06,-4.226615060278672e-05,1.296598830790147e-06,-8.392325530959837e-12,-4.731073353705690e-12,5.824045756248857e-06,7.665433296793499e-07,-2.635824886648012e-12,6.702348105091473e-12,2.692684721547833e-06,-2.961532914767940e-12,-1.807904369397134e-07,2.861597223184886e-06,-3.665913493822255e-13,6.406196878070032e-13,7.554012771618158e-07,-1.322465983945534e-05,4.413604067720572e-07,-1.261187617699493e-14,-3.040931770612132e-13,1.950543180322188e-06,2.510394501334604e-07,-3.934944231417300e-13,1.217189081867338e-14,8.489452362641597e-07,-4.439597819041652e-14,-5.554285444017802e-08,-1.002681473835283e-12,7.580706066574992e-13,-2.433476122415440e-07,-9.940387279627931e-13,-6.398199858477703e-12,7.526586687202405e-13,-1.931467713711455e-05,-1.934433717916572e-05,2.506485863756018e-13,4.285098170254750e-13,2.582714942579313e-12,-2.295698273344176e-06,-5.745550335617125e-08,7.727302112987911e-13,6.335155410340619e-13,7.181134828325723e-12,-9.849349310476842e-13,3.154196030977677e-06,2.159795030074757e-06,9.004891339318818e-13,2.380076513840880e-13,7.067417221143415e-12,4.161181375648168e-07,1.381604403062735e-06,-6.689327107004760e-13,2.583817710715252e-12,8.062030093921592e-13,3.355129027051312e-05,1.037450276494669e-06,-2.865347019743668e-12,-8.767658417154225e-12,-1.595650295874827e-12,2.629901870638632e-06,1.467862803962207e-05,3.062241338350351e-12,3.301528370477154e-12,2.835844145406309e-06,-1.304945755652620e-07,-1.269579999973428e-12,-3.508241430088591e-07,-1.558824699022471e-13,5.567501422343248e-13,1.074198138407312e-05,3.742106516676352e-07,5.635870599067412e-14,-3.112558463657280e-12,-1.302836144928140e-13,8.406987907233207e-07,4.534094558793879e-06,-4.351299146216619e-13,2.098006954695306e-14,9.394838288243557e-07,-5.907652323534100e-08,4.717137217840756e-13,-8.685242588503194e-08,-1.439593780202933e-12,-1.839880105045627e-07,-1.452566659019715e-07,1.349476156704665e-12,6.488739720750867e-08,-6.419087539468669e-09,1.921396670673221e-12,-3.952927193523762e-07,1.963136369775389e-06,-2.113016151657505e-12,-1.677316494139533e-13,1.740285164898946e-12,-5.326817064694880e-07,1.623040690594352e-05,-4.429017088613912e-12,1.517411470669737e-12,-9.946988039934424e-14,-7.449205549027475e-13,-2.158148714007891e-07,-1.014538297710892e-06,-5.762827977250308e-13,2.345343425903451e-13,-3.231644009875233e-13,2.151447347422362e-06,-1.059486511539263e-07,4.432858441693621e-14,-1.893867108527128e-12,-1.422759030817831e-14,-2.133169186387410e-06,-2.068304142876684e-05,3.939897801575818e-12,3.939439714294022e-12,7.161737362322548e-13,2.067861180189602e-05,-2.254632755051113e-06,4.304935273674192e-12,-1.490799117663132e-12,-1.631832620317266e-07,2.361411553861084e-07,4.015894667016562e-13,1.309191027562646e-07,-4.082673236391130e-13,-1.223831391510186e-12,-6.854925504083656e-07,-6.764336753551907e-06,1.958909315182536e-12,6.583443416897537e-13,2.482285920518701e-13,6.799595555885841e-06,-7.217626027045634e-07,1.954145653316321e-12,5.353602149375656e-14,-5.907570254749764e-08,7.911235539127977e-08,-2.877658280975083e-13,2.934022153926553e-08,-1.976346543645621e-12,-5.985328504749569e-08,1.858201944917358e-08,3.471299763179644e-12,-8.003262610029857e-08,-1.762995694971677e-08,1.561265195792392e-06,-1.239153713846096e-13,-2.276117380066427e-12,1.625701727401837e-06,-1.867893984056396e-05,-1.455435073238199e-05,3.105985838079533e-12,-3.152826467368456e-13,-3.650276474125772e-06,-4.327010005652714e-06,6.793162124635539e-07,6.859891260910590e-07,-3.539911390248909e-13,-8.866014873838522e-13,-5.685983788984856e-08,2.729993768719548e-07,2.558962879850571e-06,-5.108360276526171e-13,-3.995442011066669e-13,-7.148711466071049e-07,-1.017841063176445e-06,1.975004328712823e-05,-3.930847951775648e-12,-4.173961853189119e-13,-3.317185281801388e-07,5.838891079296814e-06,2.888760244684853e-06,-1.811653752332197e-12,-3.733115978741893e-12,-7.623646182247086e-07,2.486216536350796e-06,1.261760359688381e-12,-1.743670969299477e-12,-6.776029835386248e-07,-3.246123362886421e-14,-3.811900370948888e-07,6.460323445402193e-06,-1.878532937962700e-12,2.054889083705580e-13,-1.261027067428266e-07,1.851676447574772e-06,9.815431815585897e-07,-1.027609131729630e-13,-1.711745256578379e-12,-2.600784604740432e-07,8.489455840927445e-07,4.644273756151238e-14,-8.181245568364595e-13,-2.203049203017250e-07,7.368038300528654e-14,-1.230691514606490e-07,4.513933094312152e-13,2.763032338227929e-12,3.750068233584196e-08,-1.326850451567475e-12,4.392274321896050e-12,-2.019349296874489e-12,2.703208768731426e-06,-2.593100916497839e-06,-1.762106812708472e-12,-3.737118409414232e-13,-2.264046599952791e-12,-2.430018484797593e-06,-1.645905384454683e-05,-9.979783028878497e-13,6.531075145911431e-13,7.215694820211282e-12,-5.835241080571663e-13,7.698431269739178e-07,-3.125887280422728e-07,9.115619688850895e-13,5.911583898900482e-14,5.507615721380446e-12,3.686323952472489e-07,2.856554441311152e-06,-8.980946966157716e-13,-1.601574291029761e-13,2.856782500114171e-13,7.624116415521434e-06,-2.809322817614910e-06,2.993129089095212e-12,1.177737822679103e-11,4.950821101788837e-13,2.334619657944258e-06,3.033494882335127e-05,2.717824507509703e-12,3.110356185083690e-12,-1.568563240616137e-07,9.838158953415238e-08,-2.342657261530880e-12,-2.639858663415964e-06,-1.233085705499121e-13,-5.445681859702291e-13,2.265500494211015e-06,-9.689999622211999e-07,6.454275376865889e-13,3.251754293605101e-12,-1.560257252293407e-13,7.495535292314237e-07,9.719097606933423e-06,6.086152875569305e-13,-6.557496512058421e-13,-8.685264759791212e-08,2.934123480358513e-08,-7.207226288292182e-13,-8.759680599686445e-07,-6.193996578416556e-13,-4.782531749956666e-08,-1.143177072916094e-07,-9.700416119312629e-13,1.531159222341094e-07,-1.239661610873329e-08,2.842589251887614e-08,-8.431890367991884e-13,1.919148554214229e-13,-6.553750083257517e-08,-2.729216573049705e-07,3.916716170291998e-08,-3.572203797058922e-13,1.089868671164050e-12,1.040664461691172e-06,-4.325192291318300e-08,3.734623835679626e-08,4.481972547446505e-07,1.175292683898164e-12,1.916075453634659e-12,-1.310853706582694e-07,-1.786906930165786e-06,2.773608589729695e-08,-3.614661948648162e-12,-1.643434672967643e-12,1.080803276184059e-07,-7.506400665624259e-08,1.857047479021835e-08,-7.830796361690935e-13,-1.648075019536524e-13,1.864118434749718e-07,-1.420310333389845e-07,-1.963397081500057e-08,6.353620699985383e-13,1.016680761095625e-12,-4.725521791676132e-10,-1.517540655019518e-07,1.295460837558176e-11,-1.112876535501428e-12,-3.237130935570278e-07,1.195252073431468e-11,-2.659349904518967e-08,-7.416793354266656e-10,4.208908522006283e-13,4.191380497347546e-13,5.288347657331701e-08,-4.090153610422277e-08,-1.167400391194786e-08,3.455547390565093e-12,8.184055980545172e-13,-2.260852435487617e-08,-5.554361877190414e-08,1.283929612573986e-12,4.386843982973713e-13,-1.230702461291859e-07,-2.354886813904873e-13,1.320587663249872e-08,-2.728458401102025e-12,-1.144972018916422e-12,4.631884862263657e-08,-1.818845661182213e-13,-1.926720560710684e-12,-4.489191247375671e-12,-1.095527247051123e-06,5.892833814843518e-08,6.601818776519793e-13,3.213344014759864e-12,3.762768151676107e-12,1.393344109162748e-08,-1.391988891848286e-07,2.039837812788476e-13,5.785328175054309e-13,1.929360422720546e-12,2.870719243381604e-12,1.704471295758364e-06,-4.434536805370268e-08,-3.657016697470911e-12,2.359520718526618e-14,-1.378957893271197e-12,1.320791726709742e-07,2.151914810723859e-07,5.207501364118463e-12,3.640274069130787e-12,5.350400021519911e-13,1.106958874938647e-07,-8.448114904002593e-08,9.269755705176943e-13,2.337410510754430e-13,4.586442690833019e-12,-9.103932045942719e-09,2.955006686957914e-08,-1.423074440308032e-12,-1.232571505822104e-11,-4.999430123408712e-07,-9.326320737197276e-08,2.163581199434868e-12,-1.272516140156090e-07,-7.956615708752159e-12,-1.753317102949592e-13,3.215091774604632e-08,-4.781827873950288e-08,3.898970006949972e-13,6.310152966002915e-12,2.623994015778211e-13,-7.164844686428625e-09,8.421669476608958e-09,7.613406461729519e-13,2.955680400170911e-13,-1.839904174556403e-07,-5.985293137989188e-08,-3.837124265165080e-12,-4.782659324317697e-08,4.815393587105020e-13,5.163920052372062e-08,-5.720140334335062e-09,4.330314419436150e-13,-6.472437064555542e-09,4.749807200409702e-09,1.468146644060781e-12,8.016888710633652e-08,-2.788936507205507e-07,-5.942515368548355e-14,3.224778920006504e-12,-2.424831701937973e-12,2.460385150866534e-07,2.102328218396347e-08,-3.370079125281609e-13,-3.190250447610251e-13,-7.962541037083493e-13,3.142655010379011e-12,-9.028841101807345e-08,-1.818075657410074e-06,1.634602479629176e-12,6.271043611656791e-13,-2.220176599890742e-12,1.627841987604913e-06,-1.242631367231708e-07,1.662409715214274e-12,3.877310632439598e-14,8.143335308638239e-13,4.556212838058400e-08,-2.542976079153014e-08,-5.125161356975257e-13,2.246757127481203e-13,-1.338682462450292e-12,2.974757368087853e-08,-2.172367403477446e-08,8.129012985725538e-13,7.110767579484704e-13,-3.901787878454887e-07,3.569282560459366e-08,2.262104148544533e-12,-2.355544995395196e-07,1.196181820074048e-12,-3.329758272828123e-12,1.556096425326933e-08,-1.645415936625610e-09,-1.695518389198646e-12,5.869605498791241e-12,-3.759455148110759e-13,4.788720649950430e-09,-1.232080298788974e-08,1.312180561001038e-12,-1.302219306387740e-13,-1.452579833167212e-07,1.858488076699610e-08,-3.404923859168728e-12,-1.143166787894711e-07,2.107343990595633e-12,-5.716991603871932e-09,1.373135656725593e-08,-2.132982530310733e-12,-1.008885762310408e-09,-7.779954799474275e-08,-4.429540831397605e-08,-1.192503659028332e-12,6.017242257476208e-14,9.916311996034178e-09,-1.044431418853223e-06,8.428951640104628e-08,8.790348844060584e-13,2.205003109543180e-12,-1.915868649173183e-07,1.039174930946069e-08,1.842344707132082e-07,1.801915155509159e-06,-6.648786076300551e-13,-2.537653950736768e-12,2.901411560582343e-08,3.335223707228229e-07,1.744529157115033e-07,-4.528178916016487e-12,-1.617010082070779e-12,-1.716906702952201e-08,-2.708932842548436e-07,1.166006828286614e-07,-1.261661684514436e-13,1.029607495279443e-12,-4.460988916209270e-08,2.522863284208925e-08,-1.572348552366455e-07,-9.562422458137929e-14,-3.709972513779589e-13,1.663676055685830e-08,-6.507035474726220e-07,-8.696476739620729e-13,1.626892774049971e-12,1.021969824887371e-07,-1.998264587319352e-12,-1.007021961013811e-07,3.925616187617379e-08,4.556314014746382e-12,-2.131237191939250e-12,-1.501650501460866e-08,7.679674722098276e-09,-7.573796188614013e-08,6.350919156489483e-13,7.245875806935763e-12,9.669439692329367e-09,-2.433472306792527e-07,-8.384906910979117e-13,-3.134877359951832e-12,3.750458771424191e-08,8.289704221915852e-13,4.632103282295463e-08,-3.041500861001380e-12,8.806772458781813e-13,-7.121273281031898e-09,5.614275131207910e-12,-4.851625627623497e-12,-2.004912184756071e-12,1.354288221043008e-07,2.001380958337470e-08,-5.015672623039646e-14,1.664180028866011e-12,1.717225452778525e-12,-1.118653039148892e-08,-1.024362596313177e-06,6.198132473010349e-13,6.590386729707228e-14,4.426319325862532e-12,2.016961299850504e-12,-2.114844021781172e-07,-5.959668631324831e-08,3.246709563036315e-12,2.004078337782965e-12,-4.143312750294685e-12,9.780319108262983e-08,1.591367537719549e-06,-9.497788744432416e-13,4.239353722798537e-12,-6.295008230323094e-13,1.980945650678589e-10,-1.567617841334396e-07,9.025484015678412e-13,3.402365055488043e-13,3.240448004849597e-12,-4.078181636416082e-09,1.042116913033358e-07,-7.122868486444436e-13,-1.139337728944465e-11,1.724603425632368e-07,-2.398121940797733e-07,8.684592901479099e-13,4.166123031794209e-07,-3.037423046424860e-12,3.112683767909320e-13,-3.632266195452990e-09,-5.312989470396954e-08,-5.617320285573148e-13,-6.318330613313198e-12,-4.362095217574435e-12,-5.700875876399554e-09,3.364667037825697e-08,-1.390603371835982e-12,1.676688315285065e-13,6.487865343916592e-08,-8.002934713186026e-08,-1.174207350143461e-12,1.531236907177531e-07,-3.873013139789029e-12,-6.467749348215286e-09,-1.009573351681101e-09,-9.316181818802827e-13,-4.593822654083686e-08,-1.209162535920089e-08,-1.497523933410097e-12,9.749403101754816e-08,1.833073549635138e-06,-5.639213583418867e-13,5.160443023895762e-12,1.402973526801047e-12,-1.624133260962372e-06,-1.259651803731955e-07,-4.314288511812777e-13,7.912903652088182e-14,-4.489739099888820e-12,-4.791580902976034e-12,-3.268915059970114e-08,-2.064685258106573e-07,8.810145968217107e-13,-7.422131333182278e-13,-4.322754839151325e-12,1.939685803927165e-07,-3.253769997258494e-08,-5.512019396012765e-13,1.253155233601924e-11,-1.088597482242978e-12,-2.969153904523200e-07,-3.269910390819421e-07,1.931670557281659e-12,3.392613028163268e-13,1.137454854702456e-11,2.997117607654191e-07,-4.257788731862035e-09,-1.796947855009293e-12,7.888868068698543e-12,-2.303124738370790e-08,-2.660339833555569e-08,-5.072902440679740e-13,-1.712617929553156e-08,-2.262789955188081e-13,-1.169298837381086e-12,-8.540503885086326e-08,-1.202770666682536e-07,2.577832836750512e-12,-6.774499840857842e-14,1.536850860153086e-12,1.127045988209327e-07,3.428775176543731e-08,-1.949384353747848e-12,-2.930765901261435e-12,-6.413888018356251e-09,-1.762730365234810e-08,2.048498767412218e-12,-1.238289645299786e-08,-9.657092758132202e-13,4.753681944268688e-09,-7.780187810523432e-08,2.962732451260011e-12,-1.209679487346915e-08,-8.375849732234941e-09])
vect = np.array(vect, dtype=np.float32)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:06<00:00,  4.98it/s]


In [50]:
# Cambio a base natural 
rho_1 = rho_m(vect, rho_1_arrays).todense()
evals, evects = scipy.linalg.eigh(rho_1)
C = evects
vect_pro, op_pro = natural_basis_rotation(basis, C, vect)
# Calculamos la base extendida, donde vive vect_pro y restringimos
ext_basis = fixed_basis(basis.d, basis.m)
basis_pro, vect_pro = build_basis_from_vect(ext_basis, vect_pro)
# Calculamos los arrays
rho_1_arrays = rho_m_gen(basis_pro, 1, num_workers=32)
rho_2_arrays = rho_m_gen(basis_pro, 2, num_workers=32)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:29<00:00,  1.07it/s]


In [70]:
from scipy.cluster.hierarchy import fcluster, linkage
from scipy.cluster.vq import kmeans, vq
from sklearn.mixture import GaussianMixture

# Aux functions
r_eig = lambda x: np.sort(np.real(np.linalg.eigvals(x.todense())))
k = 2 # Número de clusters

def sparsity_error(A, k=2, eps=1e-7):
    sparsity_term = np.sum(np.sqrt(A**2 + eps))

    non_zero_coeffs = A[np.abs(A) > 1e-6]  
    if len(non_zero_coeffs) > 0:
        non_zero_coeffs = non_zero_coeffs.reshape(-1, 1)
        gmm = GaussianMixture(n_components=k, random_state=0)
        gmm.fit(non_zero_coeffs)
        centroids = gmm.means_.flatten()
        responsibilities = gmm.predict_proba(non_zero_coeffs)

        cluster_error = 0
        for i, coeff in enumerate(non_zero_coeffs):
            cluster_error += np.sum(responsibilities[i] * (coeff - centroids) ** 2)
    else:
        cluster_error = 0

    loss = 5 * sparsity_term + 0.5 * cluster_error
    return 1/1000 * loss


# Loss definition
def loss_fun(vect):
    # Rho 2 loss
    rho = rho_m(vect, rho_2_arrays)
    eigv = r_eig(rho)
    lrho2 = np.linalg.norm(rho_2_obj-eigv)**2
    # Rho 1 loss
    rho = rho_m(vect, rho_1_arrays)
    eigv = r_eig(rho)
    lrho1 = np.linalg.norm(rho_1_obj-eigv)**2
    # L1 + clustering loss
    l1 = sparsity_error(vect, k) 

    #print(lrho2, lrho1, l1)
    return  lrho1 + lrho2 + l1

opt = scipy.optimize.minimize(loss_fun, vect_pro, method='L-BFGS-B', options={'disp': True})

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =         1656     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  8.50158D-02    |proj g|=  1.03073D+00


 This problem is unconstrained.



At iterate    1    f=  3.99945D-02    |proj g|=  5.09660D-01

At iterate    2    f=  3.33637D-02    |proj g|=  7.40826D-02

At iterate    3    f=  3.32171D-02    |proj g|=  1.23450D-02

At iterate    4    f=  3.31702D-02    |proj g|=  6.35764D-03

At iterate    5    f=  3.30800D-02    |proj g|=  1.57559D-02

At iterate    6    f=  3.30464D-02    |proj g|=  3.90687D-03

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
 1656      6     10      1     0     0   3.907D-03   3.305D-02
  F =   3.3046434440661125E-002

STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT        


In [73]:
print(opt)
print(opt.x)
print(sparsity_error(opt.x, k=k))
print(rho_2_obj)
np.round(r_eig(rho_m(opt.x, rho_2_arrays)),2)

  message: STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
  success: False
   status: 1
      fun: 0.033046434440661125
        x: [-3.375e-09 -6.560e-09 ... -2.341e-06 -2.501e-01]
      nit: 6
      jac: [ 2.845e-08 -2.429e-08 ... -3.317e-05 -3.462e-03]
     nfev: 16570
     njev: 10
 hess_inv: <1656x1656 LbfgsInvHessProduct with dtype=float64>
[-3.37476985e-09 -6.55962993e-09 -2.46507721e-09 ... -3.19263831e-09
 -2.34107673e-06 -2.50086415e-01]
0.03302971629150185
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111 0.11111111
 0.11111111 0.11111111 0.11111111 0.33333333 0.33333333 0.33333333
 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333
 0.

array([0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11,
       0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11,
       0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.33, 0.33, 0.33, 0.33, 0.33,
       0.33, 0.33, 0.33, 0.33, 0.33, 0.33, 0.33, 0.33, 0.33, 0.33, 0.33,
       0.33, 0.33, 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 ,
       0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.5 , 0.67, 0.67, 0.67, 0.67, 0.67,
       0.67, 0.67, 0.67, 0.67, 1.  , 1.  , 1.  , 1.  , 1.  , 1.  ])

In [94]:
final_res = of.FermionOperator.zero()

tol = 1e-3
for idx, coord in enumerate(opt.x):
    final_res += basis_pro.base[idx] * coord if np.abs(coord) > tol else 0
    if np.abs(coord) > tol:
        print((1/np.abs(coord)**2))

#final_res, unique_ele(final_res)
res_round = np.sum([basis_pro.base[idx] * 1/np.sqrt(np.round(1/coord**2)) for idx, coord in enumerate(opt.x) if np.abs(coord) > tol])
vect_round = op_to_rep(basis_pro, res_round)
r_eig(rho_m(vect_round, rho_1_arrays)), rho_1_obj


15.985149014125556
15.977994228992863
145.67794611307684
144.23635285020111
36.0667153166155
36.06669576174053
144.236519674107
145.67809058988465
15.940732217777308
15.92062248719232
36.06161200056504
144.19110686510734
145.6825136168376
144.3651909188651
36.137913817300415
144.15515590265179
36.08996734303805
146.12453205723767
145.9487386480123
36.0045807486866
143.95252252055505
145.46009050066652
145.68262967629167
144.19109509086093
36.06156842879171
146.12428332738412
145.94902938020775
36.13796198870655
144.36575578324243
36.089955665595035
144.15460190992823
145.46007040730746
143.95261481327165
36.00458940655681
15.941719440781728
15.91950923819659
145.45631237154984
143.99620720608377
36.00963579334049
36.009636612591606
143.99642927360614
145.45649058176977
15.951017099193605
15.988944628712272


(array([0.49947646, 0.49947646, 0.49947646, 0.49947646, 0.4995237 ,
        0.4995237 , 0.4995237 , 0.4995237 , 0.49957093, 0.49957093,
        0.49957093, 0.49957093, 0.99904739, 0.99904739, 0.99904739,
        0.99904739]),
 array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 1. ,
        1. , 1. , 1. ]))

#### Testing

Verificamos que los eig de rho1 se preservan

In [34]:
# Cálculo de base natural
rho_1_arrays = rho_m_gen(basis, 1)
rho_1 = rho_m(vect, rho_1_arrays).todense()
evals, evects = scipy.linalg.eigh(rho_1)
C = evects

res, vect_op_ft = natural_basis_rotation(basis, C, vect)

ext_basis = fixed_basis(basis.d, basis.m)
rho_1_arrays_n = rho_m_gen(ext_basis, 1, num_workers=32)
rho_1_n = rho_m(res, rho_1_arrays_n).todense()
print(np.trace(rho_1_n), np.trace(rho_1))

evv = lambda vv: np.sort(np.real(np.linalg.eigvals(vv)))
print(evv(rho_1), evv(rho_1_n))

rep_to_op = lambda vect: np.sum([vect[i] * basis.base[i] for i in range(len(vect))])
#unique_ele(vect_op_ft), unique_ele(rep_to_op(vect))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 32/32 [00:49<00:00,  1.54s/it]


10.000000000000002 10.000000406850612
[0.         0.         0.47487306 0.47487306 0.4806496  0.4806496
 0.51935319 0.51935319 0.52513204 0.52513204 0.99999489 0.99999489
 0.99999739 0.99999739 1.00000004 1.00000004] [0.         0.         0.47487306 0.47487306 0.48064955 0.48064955
 0.51935319 0.51935319 0.525132   0.525132   0.99999485 0.99999485
 0.99999735 0.99999735 1.         1.        ]


Verificamos que la composición, es la identidad

In [94]:
res, vect_op_ft = natural_basis_rotation(basis, C, vect)
res2, vect_op_ft2 = natural_basis_rotation(ext_basis, C.T, res)

vect_op = np.sum([vect[i] * basis.base[i] for i in range(len(vect))])

of.transforms.normal_ordered(vect_op_ft2), res2, of.transforms.normal_ordered(vect_op)

(-0.009493221505521588 [11^ 10^ 9^ 8^ 7^ 4^ 3^ 2^ 1^ 0^] +
 0.06567915392863352 [11^ 10^ 9^ 8^ 7^ 5^ 4^ 2^ 1^ 0^] +
 -0.00039479040463999686 [11^ 10^ 9^ 8^ 7^ 5^ 4^ 3^ 2^ 0^] +
 -3.372243029427183e-06 [11^ 10^ 9^ 8^ 7^ 5^ 4^ 3^ 2^ 1^] +
 -0.046493134167591176 [11^ 10^ 9^ 8^ 7^ 6^ 3^ 2^ 1^ 0^] +
 -0.28426333153569405 [11^ 10^ 9^ 8^ 7^ 6^ 5^ 2^ 1^ 0^] +
 -0.00013174919476567257 [11^ 10^ 9^ 8^ 7^ 6^ 5^ 3^ 2^ 0^] +
 3.4219639201432875e-07 [11^ 10^ 9^ 8^ 7^ 6^ 5^ 3^ 2^ 1^] +
 0.002101619231354521 [12^ 10^ 9^ 8^ 7^ 5^ 3^ 2^ 1^ 0^] +
 0.2881444085777637 [12^ 10^ 9^ 8^ 7^ 6^ 4^ 2^ 1^ 0^] +
 -7.974636812387609e-05 [12^ 10^ 9^ 8^ 7^ 6^ 4^ 3^ 2^ 0^] +
 -2.402101031541447e-06 [12^ 10^ 9^ 8^ 7^ 6^ 4^ 3^ 2^ 1^] +
 0.0011760974915812092 [12^ 10^ 9^ 8^ 7^ 6^ 5^ 4^ 2^ 0^] +
 8.012654141343571e-06 [12^ 10^ 9^ 8^ 7^ 6^ 5^ 4^ 2^ 1^] +
 5.654427524408164e-07 [12^ 10^ 9^ 8^ 7^ 6^ 5^ 4^ 3^ 2^] +
 0.06567916523926123 [12^ 11^ 9^ 8^ 7^ 4^ 3^ 2^ 1^ 0^] +
 0.00389332162644885 [12^ 11^ 9^ 8^ 7^ 5^ 4^ 2^ 1^ 0^] +


Funciones auxiliares

In [99]:
def unique_ele(op):
    terms = list(op.terms.items())
    act_idx = lambda tt: [i[0] for i in tt[0]]
    terms_set = []
    for term in terms:
        if np.abs(term[1]) > 1e-6:
            terms_set.append(act_idx(term))
    return set(terms_set[0]).intersection(*terms_set[1:])

unique_ele(op_pro), unique_ele(op_fund)

(set(), set())

In [50]:
np.linalg.eigvals(rho_1)

array([0.97351211, 0.92118047, 0.31511585, 0.31511585, 0.79019157,
       0.79019157, 1.        , 0.97351211, 0.92118047, 1.        ,
       1.        , 1.        ])

In [ ]:
# Verifiquemos que los orbitales son ortogonales
op_arr = np.zeros((basis.d,basis.d),dtype=object)
sp_basis = fixed_basis(basis.d, 1)
for i in tqdm(range(basis.d)):
    for j in range(basis.d):
        oi = np.real(of.get_sparse_operator(targ_sp[i], n_qubits=basis.d))
        oj = np.real(of.get_sparse_operator(targ_sp[j], n_qubits=basis.d))
        # Sin daguear y dagueado
        assert len((oi * oj + oj * oi).data) == 0
        assert len((oi.T * oj.T + oj.T * oi.T).data) == 0
        # Términos cruzados
        op_arr[i,j] = oi * oj.T + oj.T * oi
        sh = op_arr[i,j].shape[0]
        if i != j:
            assert np.allclose(np.zeros((sh,sh)),op_arr[i,j].todense())
        else:
            assert np.allclose(np.eye(sh),op_arr[i,j].todense())

100%|██████████| 12/12 [01:00<00:00,  5.07s/it]


In [32]:
est = 0
app_r = lambda x: np.round(1/(x**2), 0)
for i, ii in enumerate(opt.x):
    if np.abs(ii) > 0.1:
        print(i, ii, basis.base[i])
        if np.abs(ii) > 0.2:
            print(app_r(ii))
            est += 1/np.sqrt(app_r(ii)) * basis.canonicals[i]
            print('')

#(1/np.sqrt(3), 1/np.sqrt(12))
#print(est)
est = 1/np.sqrt(12) * (-basis.canonicals[5]+basis.canonicals[25]+basis.canonicals[85]-basis.canonicals[105])+1/np.sqrt(3)*(basis.canonicals[45]+basis.canonicals[65])

print(r_eig(rho_m(est, rho_2_arrays)))


5 -0.27936976320734497 1.0 [0^ 1^ 2^ 3^ 4^ 7^ 8^ 9^ 12^ 13^]
13.0

25 0.28996316812175205 1.0 [0^ 1^ 2^ 3^ 5^ 7^ 8^ 9^ 11^ 13^]
12.0

45 0.5718530568911167 1.0 [0^ 1^ 2^ 3^ 6^ 7^ 8^ 9^ 11^ 12^]
3.0

65 0.572154480816062 1.0 [0^ 1^ 2^ 4^ 5^ 7^ 8^ 9^ 10^ 13^]
3.0

85 0.2897326805359033 1.0 [0^ 1^ 2^ 4^ 6^ 7^ 8^ 9^ 10^ 12^]
12.0

105 -0.27954194550717076 1.0 [0^ 1^ 2^ 5^ 6^ 7^ 8^ 9^ 10^ 11^]
13.0

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333
 0.08333333 0.08333333 0.08333333 0.08333333 0.08333333 0.33333333
 0.33333333 0.33333333 0.33333333 0.33333333 0.33333333 0.5
 0.5        0.5        0.5        0.5        0

In [259]:
r_eig(rho_m(vect, rho_2_arrays))

array([6.39649654e-18, 2.27759896e-17, 2.27759896e-17, 2.97340303e-17,
       1.05847838e-08, 1.05847839e-08, 8.12469946e-02, 8.12469946e-02,
       8.12469946e-02, 8.12469946e-02, 8.12469946e-02, 8.12469946e-02,
       8.26670825e-02, 8.26670825e-02, 8.26670825e-02, 8.26670825e-02,
       8.26670825e-02, 8.26670825e-02, 3.36085923e-01, 3.36085923e-01,
       3.36085923e-01, 3.36085923e-01, 3.36085923e-01, 3.36085923e-01,
       7.48001237e-01, 7.48001237e-01, 7.51998753e-01, 7.51998753e-01])

#### Testing

In [256]:
vect = [-4.646928018967011e-01,2.770294629368069e-13,6.042253100720720e-13,-6.024516357133285e-13,-2.567187786718633e-13,1.437594192304828e-01,-2.763730452330614e-13,4.646928018967681e-01,6.783135520862164e-13,6.755585098225312e-13,-1.437594192304206e-01,2.565069368880155e-13,-6.036534951757788e-13,-6.777043000828106e-13,-6.255919748617283e-14,-2.875188384604817e-01,-6.292320178844996e-13,-5.606330583371935e-13,6.014298958703158e-13,-6.759194003610720e-13,-2.875188384604817e-01,-6.207393804663961e-14,-6.269833844031960e-13,5.590088393571927e-13,2.557458444682729e-13,-1.437594192304207e-01,6.287588334290853e-13,6.270036860946324e-13,4.000749631766127e-01,-2.383049928519640e-13,1.437594192304827e-01,-2.559215089660516e-13,5.600763473792744e-13,-5.580405389843103e-13,2.382296361695720e-13,-4.000749631765536e-01]
vect = np.array(vect)
vect

array([-4.64692802e-01,  2.77029463e-13,  6.04225310e-13, -6.02451636e-13,
       -2.56718779e-13,  1.43759419e-01, -2.76373045e-13,  4.64692802e-01,
        6.78313552e-13,  6.75558510e-13, -1.43759419e-01,  2.56506937e-13,
       -6.03653495e-13, -6.77704300e-13, -6.25591975e-14, -2.87518838e-01,
       -6.29232018e-13, -5.60633058e-13,  6.01429896e-13, -6.75919400e-13,
       -2.87518838e-01, -6.20739380e-14, -6.26983384e-13,  5.59008839e-13,
        2.55745844e-13, -1.43759419e-01,  6.28758833e-13,  6.27003686e-13,
        4.00074963e-01, -2.38304993e-13,  1.43759419e-01, -2.55921509e-13,
        5.60076347e-13, -5.58040539e-13,  2.38229636e-13, -4.00074963e-01])

In [167]:
np.sort(np.linalg.eigvals(rho_m(vect, rho_2_arrays).todense()))

array([6.39649654e-18+0.00000000e+00j, 2.27759896e-17-1.60872919e-17j,
       2.27759896e-17+1.60872919e-17j, 2.97340303e-17+0.00000000e+00j,
       1.05847838e-08+0.00000000e+00j, 1.05847839e-08+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.12469946e-02+0.00000000e+00j, 8.12469946e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       8.26670825e-02+0.00000000e+00j, 8.26670825e-02+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       3.36085923e-01+0.00000000e+00j, 3.36085923e-01+0.00000000e+00j,
       7.48001237e-01+0.00000000e+00j, 7.48001237e-01+0.00000000e+00j,
       7.51998753e-01+0.00000000e+00j, 7.51998753e-01+0.00000000e+00j])